## Explore data 

In [5]:
import pandas as pd
import numpy as np

In [6]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
submission = pd.read_csv('../data/sample_submission.csv')
age_gender = pd.read_csv('../data/age_gender_info.csv')

In [7]:
train.shape, test.shape

((2952, 15), (1022, 14))

### 잘못된 train 데이터 제외하기

In [8]:
err = ['C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988' ,'C2431', 'C1649', 'C2085', 'C1397' ,'C1036']
train = train.query(' 단지코드 not in @err')

In [11]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수'],
      dtype='object')

In [12]:
test.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수'],
      dtype='object')

In [13]:
# 컬럼명 변경 - 지하쳘역 수와 버스 정류장 수의 컬럼명을 지하철, 버스로 변경
train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '자격유형',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]

test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '자격유형',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

In [14]:
# 임대보증금과 임대료의 타입변경이 필요함
train.dtypes

단지코드         object
총세대수          int64
임대건물구분       object
지역           object
공급유형         object
전용면적        float64
전용면적별세대수      int64
공가수         float64
자격유형         object
임대보증금        object
임대료          object
지하철         float64
버스          float64
단지내주차면수     float64
등록차량수       float64
dtype: object

In [15]:
# 임대보증금과 임대료의 타입변경이 필요함
test.dtypes

단지코드         object
총세대수          int64
임대건물구분       object
지역           object
공급유형         object
전용면적        float64
전용면적별세대수      int64
공가수         float64
자격유형         object
임대보증금        object
임대료          object
지하철         float64
버스          float64
단지내주차면수     float64
dtype: object

In [16]:
train['임대보증금'] = pd.to_numeric(train['임대보증금'], errors='coerce')
train['임대료'] = pd.to_numeric(train['임대료'], errors='coerce')

test['임대보증금'] = pd.to_numeric(test['임대보증금'], errors='coerce')
test['임대료'] = pd.to_numeric(test['임대료'], errors='coerce')

In [17]:
train.임대건물구분.unique()

array(['아파트', '상가'], dtype=object)

In [13]:
train.지역.unique()

array(['경상남도', '대전광역시', '경기도', '전라북도', '강원도', '광주광역시', '충청남도', '부산광역시',
       '제주특별자치도', '울산광역시', '충청북도', '전라남도', '경상북도', '대구광역시', '서울특별시',
       '세종특별자치시'], dtype=object)

In [14]:
train.공급유형.unique()

array(['국민임대', '공공임대(50년)', '영구임대', '임대상가', '공공임대(10년)', '공공임대(분납)',
       '장기전세', '공공분양', '행복주택', '공공임대(5년)'], dtype=object)

In [15]:
train.자격유형.unique()

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O'], dtype=object)

## 결측치 확인

In [16]:
train.isna().sum()

단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          0
임대보증금       581
임대료         584
지하철         207
버스            0
단지내주차면수       0
등록차량수         0
dtype: int64

In [17]:
test.isna().sum()

단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          2
임대보증금       186
임대료         186
지하철          42
버스            0
단지내주차면수       0
dtype: int64

## 자격유형 결측치 처리

In [18]:
test.자격유형.value_counts()

A    572
D    180
H     92
J     84
C     34
K     16
L     12
N     10
E     10
I      7
M      2
G      1
Name: 자격유형, dtype: int64

In [19]:
test[test.자격유형.isna()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000.0,37470.0,0.0,2.0,840.0
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000.0,44770.0,0.0,2.0,173.0


In [20]:
# 같은 단지에 비해 임대보증금이 월등히 높고, 임대료가 낮음
# 같은 단지와 자격유형이 같거나 높다고 볼 수 있음
test[test.단지코드 == 'C2411'].head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
193,C2411,962,아파트,경상남도,국민임대,39.43,56,25.0,A,11992000.0,100720.0,0.0,2.0,840.0
194,C2411,962,아파트,경상남도,국민임대,39.72,336,25.0,A,11992000.0,100720.0,0.0,2.0,840.0
195,C2411,962,아파트,경상남도,국민임대,39.82,179,25.0,A,11992000.0,100720.0,0.0,2.0,840.0
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000.0,37470.0,0.0,2.0,840.0
197,C2411,962,아파트,경상남도,국민임대,51.93,150,25.0,A,21586000.0,171480.0,0.0,2.0,840.0


In [21]:
# 'A'로 대체
test.loc[test.단지코드.isin(['C2411']) & test.자격유형.isnull(), '자격유형'] = 'A'

In [22]:
# 같은 단지에 비해 임대보증금과 임대료가 차이가 미미함.
# 같은 단지와 자격유형이 같다고 볼 수 있음
test[test.단지코드 == 'C2253'].head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000.0,44770.0,0.0,2.0,173.0
259,C2253,1161,아파트,강원도,영구임대,31.32,239,0.0,C,3731000.0,83020.0,0.0,2.0,173.0
260,C2253,1161,아파트,강원도,영구임대,31.32,149,0.0,C,3731000.0,83020.0,0.0,2.0,173.0
261,C2253,1161,상가,강원도,임대상가,13.77,1,0.0,D,NaN,NaN,0.0,2.0,173.0
262,C2253,1161,상가,강원도,임대상가,22.89,1,0.0,D,NaN,NaN,0.0,2.0,173.0


In [23]:
#'C'로 대체
test.loc[test.단지코드.isin(['C2253']) & test.자격유형.isnull(), '자격유형'] = 'C'

### 임대상가의 임대보증금, 임대료 결측치 처리 [train]
- 'C1874', 'C2416', 'C2621', 'C1616', 'C1704', 'C2258','C2038', 'C1859'(상가 결측치 성규 8개)
- 'C1350', 'C1326', 'C1786', 'C2186 (아파트 결측치 전체 4개)

In [24]:
pd.options.display.float_format = '{:.2f}'.format

In [25]:
train[train.임대보증금.isna()].단지코드.unique()

array(['C1925', 'C1874', 'C2416', 'C2621', 'C1616', 'C1704', 'C2258',
       'C2038', 'C1859', 'C1722', 'C2190', 'C1476', 'C1983', 'C2135',
       'C2034', 'C1109', 'C2289', 'C2597', 'C2310', 'C2132', 'C1439',
       'C1899', 'C1056', 'C2644', 'C1206', 'C1775', 'C1790', 'C2109',
       'C1698', 'C1004', 'C1875', 'C2212', 'C2571', 'C1350', 'C1326',
       'C1786', 'C2186'], dtype=object)

In [26]:
train[(train.임대건물구분=='상가') & (train.임대보증금.isna())].단지코드.unique()

array(['C1925', 'C1874', 'C2416', 'C2621', 'C1616', 'C1704', 'C2258',
       'C2038', 'C1859', 'C1722', 'C2190', 'C1476', 'C1983', 'C2135',
       'C2034', 'C1109', 'C2289', 'C2597', 'C2310', 'C2132', 'C1439',
       'C1899', 'C1056', 'C2644', 'C1206', 'C1775', 'C1790', 'C2109',
       'C1698', 'C1004', 'C1875', 'C2212', 'C2571'], dtype=object)

In [27]:
train[(train.임대건물구분=='아파트') & (train.임대보증금.isna())].단지코드.unique()

array(['C1350', 'C1326', 'C1786', 'C2186'], dtype=object)

In [28]:
train[train.단지코드 == 'C1925']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
86,C1925,601,아파트,강원도,영구임대,26.37,298,9.00,C,8052000.00,130850.00,0.00,4.00,117.00,75.00
87,C1925,601,아파트,강원도,영구임대,31.32,298,9.00,C,9174000.00,144460.00,0.00,4.00,117.00,75.00
88,C1925,601,상가,강원도,임대상가,32.10,1,9.00,D,nan,nan,0.00,4.00,117.00,75.00
89,C1925,601,상가,강원도,임대상가,32.10,1,9.00,D,nan,nan,0.00,4.00,117.00,75.00
90,C1925,601,상가,강원도,임대상가,32.10,1,9.00,D,nan,nan,0.00,4.00,117.00,75.00
91,C1925,601,상가,강원도,임대상가,72.16,1,9.00,D,nan,nan,0.00,4.00,117.00,75.00


In [29]:
#32.10
train.loc[88:90,'임대보증금']=15984000
train.loc[88:90,'임대료']=380000

#72.16
train.loc[91:91,'임대보증금']=14180000
train.loc[91:91,'임대료']=330800

In [30]:
train[train.단지코드 == 'C1874']
#공주옥룡1
#https://www.kohom.or.kr/moffice/1101001/MM002004.do?page=1

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
98,C1874,619,아파트,충청남도,영구임대,26.37,294,2.00,C,3141000.00,69900.00,nan,2.00,97.00,62.00
99,C1874,619,아파트,충청남도,영구임대,26.37,149,2.00,C,3141000.00,69900.00,nan,2.00,97.00,62.00
100,C1874,619,아파트,충청남도,영구임대,31.32,149,2.00,C,3731000.00,83020.00,nan,2.00,97.00,62.00
101,C1874,619,상가,충청남도,임대상가,12.62,1,2.00,D,nan,nan,nan,2.00,97.00,62.00
102,C1874,619,상가,충청남도,임대상가,17.40,1,2.00,D,nan,nan,nan,2.00,97.00,62.00
103,C1874,619,상가,충청남도,임대상가,17.40,1,2.00,D,nan,nan,nan,2.00,97.00,62.00
104,C1874,619,상가,충청남도,임대상가,22.89,1,2.00,D,nan,nan,nan,2.00,97.00,62.00
105,C1874,619,상가,충청남도,임대상가,23.13,1,2.00,D,nan,nan,nan,2.00,97.00,62.00
106,C1874,619,상가,충청남도,임대상가,23.13,1,2.00,D,nan,nan,nan,2.00,97.00,62.00
107,C1874,619,상가,충청남도,임대상가,23.25,1,2.00,D,nan,nan,nan,2.00,97.00,62.00


In [31]:
train[(train.단지코드 == 'C1874') & (train.공급유형 == '임대상가')]['전용면적'].unique()

array([12.62, 17.4 , 22.89, 23.13, 23.25, 27.75, 28.19, 34.8 , 42.35,
       42.4 , 49.37, 55.17, 55.5 ])

In [32]:
#12.62(109호 문화슈퍼)
train.loc[101:101,'임대보증금']=4280000
train.loc[101:101,'임대료']=107900

#17.40(111호 문화슈퍼)
train.loc[102:102,'임대보증금']=1856000
train.loc[102:102,'임대료']=51570

#17.40(112호 문화슈퍼)
train.loc[103:103,'임대보증금']=5139000
train.loc[103:103,'임대료']=126670

#22.89(114호 천지인부동산)
train.loc[104:104,'임대보증금']=8896000
train.loc[104:104,'임대료']=249680

#23.13(104호 근린생활시설)
train.loc[105:105,'임대보증금']=8199000
train.loc[105:105,'임대료']=227760

#23.13(105호 주공기름집)
train.loc[105:105,'임대보증금']=7560000
train.loc[105:105,'임대료']=190500

#27.75(102호 문화슈퍼)
train.loc[108:108,'임대보증금']=2951000
train.loc[108:108,'임대료']=81990

#27.75(103호 근린생활시설)
train.loc[109:109,'임대보증금']=9839000
train.loc[109:109,'임대료']=273310

#27.75(106호, 107호 가야촌옥룡점)
train.loc[110:111,'임대보증금']=8810000
train.loc[110:111,'임대료']=222200

#27.75(202호 이삿짐센터)
train.loc[112:112,'임대보증금']=4094000
train.loc[112:112,'임대료']=112380

#27.75(203호 그린이용원, 204호 공주시세탁협회, 205호 주은건설, 206호 유일개발)
train.loc[113:116,'임대보증금']=4310000
train.loc[113:116,'임대료']=107600

#27.75(210호 경비휴게실)
train.loc[117:117,'임대보증금']=4410000
train.loc[117:117,'임대료']=110000

#27.75(211호 유현종합건설)
train.loc[118:118,'임대보증금']=4390000
train.loc[118:118,'임대료']=109700

#28.19(201호 비어비어)
train.loc[119:119,'임대보증금']=1586000
train.loc[119:119,'임대료']=44080

#34.8(110호 문화슈퍼)
train.loc[120:120,'임대보증금']=11370000
train.loc[120:120,'임대료']=286600

#34.8(113호 주공기름집)
train.loc[121:121,'임대보증금']=3697000
train.loc[121:121,'임대료']=102710

#42.35(213호 프라임건설)
train.loc[122:122,'임대보증금']=6811000
train.loc[122:122,'임대료']=158920

#42.4(212호 비어비어)
train.loc[123:123,'임대보증금']=7180000
train.loc[123:123,'임대료']=179200

#49.37(101호 문화슈퍼)
train.loc[124:124,'임대보증금']=16440000
train.loc[124:124,'임대료']=414400

#55.17(207호 작은도서관, 208호 솔미용실)
train.loc[125:126,'임대보증금']=8910000
train.loc[125:126,'임대료']=222400

#55.5(209호 길세탁소)
train.loc[127:127,'임대보증금']=8790000
train.loc[127:127,'임대료']=219400

#완벽

In [33]:
#못찾은 것은 찾은 것들의 평균으로 대치
#train.loc[train.단지코드.isin(['C1874']) & train.임대보증금.isnull(), '임대보증금'] = train.loc[train.단지코드.isin(['C1874']) & (train.공급유형 == '임대상가')]['임대보증금'].mean()
#train.loc[train.단지코드.isin(['C1874']) & train.임대료.isnull(), '임대료'] = train.loc[train.단지코드.isin(['C1874']) & (train.공급유형 == '임대상가')]['임대료'].mean()

In [34]:
train[train.단지코드 == 'C2416']
#김해구산1
#https://www.kohom.or.kr/moffice/1304001/MM002004.do

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
131,C2416,560,아파트,경상남도,영구임대,26.34,360,0.00,C,3138000.00,69820.00,0.00,5.00,154.00,63.00
132,C2416,560,아파트,경상남도,영구임대,30.48,192,0.00,C,3631000.00,80800.00,0.00,5.00,154.00,63.00
133,C2416,560,상가,경상남도,임대상가,19.69,1,0.00,D,nan,nan,0.00,5.00,154.00,63.00
134,C2416,560,상가,경상남도,임대상가,27.12,1,0.00,D,nan,nan,0.00,5.00,154.00,63.00
135,C2416,560,상가,경상남도,임대상가,32.54,1,0.00,D,nan,nan,0.00,5.00,154.00,63.00
136,C2416,560,상가,경상남도,임대상가,36.43,1,0.00,D,nan,nan,0.00,5.00,154.00,63.00
137,C2416,560,상가,경상남도,임대상가,46.89,1,0.00,D,nan,nan,0.00,5.00,154.00,63.00
138,C2416,560,상가,경상남도,임대상가,49.99,1,0.00,D,nan,nan,0.00,5.00,154.00,63.00
139,C2416,560,상가,경상남도,임대상가,51.14,1,0.00,D,nan,nan,0.00,5.00,154.00,63.00
140,C2416,560,상가,경상남도,임대상가,317.17,1,0.00,D,nan,nan,0.00,5.00,154.00,63.00


In [35]:
train[(train.단지코드 == 'C2416') & (train.공급유형 == '임대상가')]['전용면적'].unique()

array([ 19.69,  27.12,  32.54,  36.43,  46.89,  49.99,  51.14, 317.17])

In [36]:
#19.69(105호)
train.loc[133:133,'임대보증금']=7520000
train.loc[133:133,'임대료']=177500

#27.12(106호)
train.loc[134:134,'임대보증금']=10630000
train.loc[134:134,'임대료']=271500

#32.54(103호)
train.loc[135:135,'임대보증금']=4540000
train.loc[135:135,'임대료']=114200

#36.43(104호)
train.loc[136:136,'임대보증금']=7070000
train.loc[136:136,'임대료']=167100

#46.89(102호)
train.loc[137:137,'임대보증금']=10020000
train.loc[137:137,'임대료']=236600

#49.99(107호)
train.loc[138:138,'임대보증금']=6750000
train.loc[138:138,'임대료']=158400

#51.14(101호)
train.loc[139:139,'임대보증금']=6080000
train.loc[139:139,'임대료']=154100

#317.17(지하 복지관)
train.loc[140:140,'임대보증금']=2000000
train.loc[140:140,'임대료']=0

#완벽

In [37]:
#못찾은 것은 찾은 것들의 평균으로 대치
#train.loc[train.단지코드.isin(['C2416']) & train.임대보증금.isnull(), '임대보증금'] = train.loc[train.단지코드.isin(['C2416']) & (train.공급유형 == '임대상가')]['임대보증금'].mean()
#train.loc[train.단지코드.isin(['C2416']) & train.임대료.isnull(), '임대료'] = train.loc[train.단지코드.isin(['C2416']) & (train.공급유형 == '임대상가')]['임대료'].mean()

In [38]:
train[train.단지코드 == 'C2621']
#대전둔산3
#https://www.kohom.or.kr/moffice/0603003/MM002004.do

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
153,C2621,1396,아파트,대전광역시,영구임대,26.37,890,26.00,C,5787000.00,79980.00,1.00,4.00,277.00,133.00
154,C2621,1396,아파트,대전광역시,영구임대,26.37,141,26.00,C,5787000.00,79980.00,1.00,4.00,277.00,133.00
155,C2621,1396,아파트,대전광역시,영구임대,26.37,20,26.00,C,5787000.00,79980.00,1.00,4.00,277.00,133.00
156,C2621,1396,아파트,대전광역시,영구임대,26.37,20,26.00,C,5787000.00,79980.00,1.00,4.00,277.00,133.00
157,C2621,1396,아파트,대전광역시,영구임대,31.32,298,26.00,C,6873000.00,94990.00,1.00,4.00,277.00,133.00
158,C2621,1396,아파트,대전광역시,영구임대,52.74,9,26.00,C,11574000.00,159960.00,1.00,4.00,277.00,133.00
159,C2621,1396,아파트,대전광역시,영구임대,52.74,4,26.00,C,11574000.00,159960.00,1.00,4.00,277.00,133.00
160,C2621,1396,아파트,대전광역시,영구임대,52.74,4,26.00,C,11574000.00,159960.00,1.00,4.00,277.00,133.00
161,C2621,1396,상가,대전광역시,임대상가,31.84,1,26.00,D,nan,nan,1.00,4.00,277.00,133.00
162,C2621,1396,상가,대전광역시,임대상가,31.84,1,26.00,D,nan,nan,1.00,4.00,277.00,133.00


In [39]:
train[(train.단지코드 == 'C2621') & (train.공급유형 == '임대상가')]['전용면적'].unique()

array([ 31.84,  63.68, 137.49])

In [40]:
#31.84(101호 병원)
train.loc[161:161,'임대보증금']=21930000
train.loc[161:161,'임대료']=567600

#31.84(102호 미용실, 104호 슈퍼)
train.loc[162:163,'임대보증금']=21330000
train.loc[162:163,'임대료']=552100

#31.84(103호 약국)
train.loc[164:164,'임대보증금']=27260000
train.loc[164:164,'임대료']=710800

#31.84(105호 슈퍼)
train.loc[165:165,'임대보증금']=22460000
train.loc[165:165,'임대료']=581200

#31.84(201호 철학관)
train.loc[166:166,'임대보증금']=8860000
train.loc[166:166,'임대료']=227300

#31.84(202호 미용실, 203호 세탁소)
train.loc[167:168,'임대보증금']=8960000
train.loc[167:168,'임대료']=229900

#63.68(204호 병원)
train.loc[169:169,'임대보증금']=19000000
train.loc[169:169,'임대료']=491100

#137.49(지하 1층 창고)
train.loc[170:170,'임대보증금']=37710000
train.loc[170:170,'임대료']=886900

#완벽

In [41]:
train[train.단지코드 == 'C1616']
#대전법동3 임대상가
#https://www.kohom.or.kr/moffice/0601001/MM002004.do

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
176,C1616,1507,아파트,대전광역시,영구임대,26.37,588,1.00,C,5787000.00,79980.00,nan,2.00,407.00,98.00
177,C1616,1507,아파트,대전광역시,영구임대,31.32,450,1.00,C,6873000.00,94990.00,nan,2.00,407.00,98.00
178,C1616,1507,아파트,대전광역시,영구임대,40.32,450,1.00,E,54040000.00,28140.00,nan,2.00,407.00,98.00
179,C1616,1507,상가,대전광역시,임대상가,38.00,1,1.00,D,nan,nan,nan,2.00,407.00,98.00
180,C1616,1507,상가,대전광역시,임대상가,38.00,1,1.00,D,nan,nan,nan,2.00,407.00,98.00
181,C1616,1507,상가,대전광역시,임대상가,37.26,1,1.00,D,nan,nan,nan,2.00,407.00,98.00
182,C1616,1507,상가,대전광역시,임대상가,37.41,1,1.00,D,nan,nan,nan,2.00,407.00,98.00
183,C1616,1507,상가,대전광역시,임대상가,37.41,1,1.00,D,nan,nan,nan,2.00,407.00,98.00
184,C1616,1507,상가,대전광역시,임대상가,37.41,1,1.00,D,nan,nan,nan,2.00,407.00,98.00
185,C1616,1507,상가,대전광역시,임대상가,37.49,1,1.00,D,nan,nan,nan,2.00,407.00,98.00


In [42]:
train[(train.단지코드 == 'C1616') & (train.공급유형 == '임대상가')]['전용면적'].unique()

array([ 38.  ,  37.26,  37.41,  37.49,  37.95,  38.04,  39.33,  54.51,
        54.61,  54.91,  75.98, 109.11, 583.4 ])

In [43]:
#38.00(106호 로또방)
train.loc[179:179,'임대보증금']=16480000
train.loc[179:179,'임대료']=744080

#38.00(110호 25시주공마트)
train.loc[180:180,'임대보증금']=15296000
train.loc[180:180,'임대료']=594320

#37.26(101호 으뜸문구)
train.loc[181:181,'임대보증금']=24020000
train.loc[181:181,'임대료']=611700

#37.41(102호 주공하이퍼야채, 103호 주공화장품)
train.loc[182:183,'임대보증금']=20880000
train.loc[182:183,'임대료']=531600

#37.41(104호 사라(옷))
train.loc[184:184,'임대보증금']=14600000
train.loc[184:184,'임대료']=657900

#37.49(105호 장태순헤어샵)
train.loc[185:185,'임대보증금']=19590000
train.loc[185:185,'임대료']=544180

#37.95(202호 근린생활시설)
train.loc[186:186,'임대보증금']=8227000
train.loc[186:186,'임대료']=228530

#37.95(203호 현대옷수선)
train.loc[187:187,'임대보증금']=5600000
train.loc[187:187,'임대료']=235000

#38.04(204호 엄마손밥상)
train.loc[188:188,'임대보증금']=9120000
train.loc[188:188,'임대료']=233900

#39.33(107호 우리연쇄점)
train.loc[189:189,'임대보증금']=15400000
train.loc[189:189,'임대료']=693900

#39.33(108호 베이커리, 109호 과일스토리)
train.loc[190:191,'임대보증금']=22020000
train.loc[190:191,'임대료']=560700

#54.51(207호 여왕미용실), 54.61(206호 조이미용실)
train.loc[192:193,'임대보증금']=11680000
train.loc[192:193,'임대료']=274800

#54.91(205호 은혜세탁소)
train.loc[194:194,'임대보증금']=12590000
train.loc[194:194,'임대료']=292700

#75.98(201호 은강한의언)
train.loc[195:195,'임대보증금']=11400000
train.loc[195:195,'임대료']=478600

#109.11(208호 새싹지역아동센터)
train.loc[196:196,'임대보증금']=16400000
train.loc[196:196,'임대료']=227280

#583.4(지하 근린생활시설)
train.loc[197:197,'임대보증금']=67261000
train.loc[197:197,'임대료']=1868370

#완벽

In [44]:
train[train.단지코드 == 'C1704'] 
#대전산내
#https://www.kohom.or.kr/web/mainComm/HM001004.do?mode=view&schDo=0600000&schCon=0&schStr=&page=1&p_idx=12202

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
198,C1704,639,아파트,대전광역시,영구임대,39.30,204,34.00,C,8625000.00,119190.00,nan,7.00,162.00,131.00
199,C1704,639,아파트,대전광역시,영구임대,39.69,420,34.00,C,8710000.00,120370.00,nan,7.00,162.00,131.00
200,C1704,639,상가,대전광역시,임대상가,31.84,1,34.00,D,nan,nan,nan,7.00,162.00,131.00
201,C1704,639,상가,대전광역시,임대상가,31.84,1,34.00,D,nan,nan,nan,7.00,162.00,131.00
202,C1704,639,상가,대전광역시,임대상가,31.84,1,34.00,D,nan,nan,nan,7.00,162.00,131.00
203,C1704,639,상가,대전광역시,임대상가,31.84,1,34.00,D,nan,nan,nan,7.00,162.00,131.00
204,C1704,639,상가,대전광역시,임대상가,31.84,1,34.00,D,nan,nan,nan,7.00,162.00,131.00
205,C1704,639,상가,대전광역시,임대상가,31.84,1,34.00,D,nan,nan,nan,7.00,162.00,131.00
206,C1704,639,상가,대전광역시,임대상가,31.84,1,34.00,D,nan,nan,nan,7.00,162.00,131.00
207,C1704,639,상가,대전광역시,임대상가,31.84,1,34.00,D,nan,nan,nan,7.00,162.00,131.00


In [45]:
train[(train.단지코드 == 'C1704') & (train.공급유형 == '임대상가')]['전용면적'].unique()

array([ 31.84, 126.65])

In [46]:
#31.84(101호, 102호)
train.loc[200:201,'임대보증금']=10230000
train.loc[200:201,'임대료']=284180

#31.84(103호)
train.loc[202:202,'임대보증금']=12050000
train.loc[202:202,'임대료']=248500

#31.84(104호, 105호)
train.loc[203:204,'임대보증금']=10945000
train.loc[203:204,'임대료']=304050

#31.84(106호)
train.loc[205:205,'임대보증금']=8600000
train.loc[205:205,'임대료']=272000

#31.84(107호)
train.loc[206:206,'임대보증금']=9600000
train.loc[206:206,'임대료']=415500

#31.84(201호)
train.loc[207:207,'임대보증금']=3200000
train.loc[207:207,'임대료']=129000

#31.84(202호)
train.loc[208:208,'임대보증금']=4522000
train.loc[208:208,'임대료']=86450

#31.84(203호~207호)
train.loc[209:213,'임대보증금']=3767000
train.loc[209:213,'임대료']=104660

#126.65(지하1층)
train.loc[214:214,'임대보증금']=9205000
train.loc[214:214,'임대료']=255690

#완벽

In [47]:
train[train.단지코드 == 'C2258']
#대전중촌2
#주택관리공단에 임대상가 정보 없음.

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
215,C2258,965,아파트,대전광역시,영구임대,26.37,588,13.00,C,5787000.00,79980.00,nan,3.00,287.00,78.00
216,C2258,965,아파트,대전광역시,영구임대,31.32,180,13.00,C,6873000.00,94990.00,nan,3.00,287.00,78.00
217,C2258,965,아파트,대전광역시,영구임대,40.32,180,13.00,C,8848000.00,122290.00,nan,3.00,287.00,78.00
218,C2258,965,상가,대전광역시,임대상가,19.00,1,13.00,D,nan,nan,nan,3.00,287.00,78.00
219,C2258,965,상가,대전광역시,임대상가,14.10,1,13.00,D,nan,nan,nan,3.00,287.00,78.00
220,C2258,965,상가,대전광역시,임대상가,19.31,1,13.00,D,nan,nan,nan,3.00,287.00,78.00
221,C2258,965,상가,대전광역시,임대상가,21.19,1,13.00,D,nan,nan,nan,3.00,287.00,78.00
222,C2258,965,상가,대전광역시,임대상가,21.19,1,13.00,D,nan,nan,nan,3.00,287.00,78.00
223,C2258,965,상가,대전광역시,임대상가,22.95,1,13.00,D,nan,nan,nan,3.00,287.00,78.00
224,C2258,965,상가,대전광역시,임대상가,23.40,1,13.00,D,nan,nan,nan,3.00,287.00,78.00


In [48]:
train[(train.단지코드 == 'C2258') & (train.공급유형 == '임대상가')]['전용면적'].unique()

array([ 19.  ,  14.1 ,  19.31,  21.19,  22.95,  23.4 ,  27.23,  31.85,
        32.29,  35.13,  36.47,  50.08, 240.22])

In [49]:
#19.00
train.loc[218:218,'임대보증금']=2600000
train.loc[218:218,'임대료']=107900

#14.10
train.loc[219:219,'임대보증금']=2456000
train.loc[219:219,'임대료']=68230

#19.31

#21.19
train.loc[221:222,'임대보증금']=7800000
train.loc[221:222,'임대료']=357600

#22.95
train.loc[223:223,'임대보증금']=3000000
train.loc[223:223,'임대료']=124500

#23.40(106호)
train.loc[224:224,'임대보증금']=14240000
train.loc[224:224,'임대료']=341760

#23.40(107호)
train.loc[225:225,'임대보증금']=12150000
train.loc[225:225,'임대료']=337500

#27.23
train.loc[228:228,'임대보증금']=3230000
train.loc[228:228,'임대료']=134230

#31.85

#32.29
train.loc[230:230,'임대보증금']=4200000
train.loc[230:230,'임대료']=174700

#35.13
train.loc[231:231,'임대보증금']=4180000
train.loc[231:231,'임대료']=173750

#36.47

#50.08

#240.22
train.loc[234:234,'임대보증금']=28593000
train.loc[234:234,'임대료']=794250

In [50]:
#못찾은 것은 찾은 것들의 평균으로 대치
train.loc[train.단지코드.isin(['C2258']) & train.임대보증금.isnull(), '임대보증금'] = train.loc[train.단지코드.isin(['C2258']) & (train.공급유형 == '임대상가')]['임대보증금'].mean()
train.loc[train.단지코드.isin(['C2258']) & train.임대료.isnull(), '임대료'] = train.loc[train.단지코드.isin(['C2258']) & (train.공급유형 == '임대상가')]['임대료'].mean()

In [51]:
train[train.단지코드 == 'C2038']
#대전판암3
#https://www.kohom.or.kr/moffice/0602001/MM002004.do

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
241,C2038,785,아파트,대전광역시,영구임대,26.37,588,7.00,C,5787000.00,79980.00,1.00,10.00,150.00,108.00
242,C2038,785,아파트,대전광역시,영구임대,31.32,90,7.00,C,6873000.00,94990.00,1.00,10.00,150.00,108.00
243,C2038,785,아파트,대전광역시,영구임대,40.32,90,7.00,C,8848000.00,122290.00,1.00,10.00,150.00,108.00
244,C2038,785,상가,대전광역시,임대상가,19.00,1,7.00,D,nan,nan,1.00,10.00,150.00,108.00
245,C2038,785,상가,대전광역시,임대상가,14.10,1,7.00,D,nan,nan,1.00,10.00,150.00,108.00
246,C2038,785,상가,대전광역시,임대상가,19.31,1,7.00,D,nan,nan,1.00,10.00,150.00,108.00
247,C2038,785,상가,대전광역시,임대상가,21.19,1,7.00,D,nan,nan,1.00,10.00,150.00,108.00
248,C2038,785,상가,대전광역시,임대상가,21.19,1,7.00,D,nan,nan,1.00,10.00,150.00,108.00
249,C2038,785,상가,대전광역시,임대상가,22.95,1,7.00,D,nan,nan,1.00,10.00,150.00,108.00
250,C2038,785,상가,대전광역시,임대상가,23.40,1,7.00,D,nan,nan,1.00,10.00,150.00,108.00


In [52]:
train[(train.단지코드 == 'C2038') & (train.공급유형 == '임대상가')]['전용면적'].unique()

array([ 19.  ,  14.1 ,  19.31,  21.19,  22.95,  23.4 ,  27.23,  31.85,
        32.29,  35.13,  36.47,  50.08, 240.22])

In [53]:
train[(train.단지코드 == 'C2038') & (train.공급유형 == '임대상가')]['전용면적'].value_counts()

23.40     4
21.19     2
32.29     1
50.08     1
31.85     1
35.13     1
240.22    1
22.95     1
27.23     1
14.10     1
36.47     1
19.31     1
19.00     1
Name: 전용면적, dtype: int64

In [54]:
#19.00
train.loc[244,'임대보증금']=2884000
train.loc[244,'임대료']=71800

#14.10
train.loc[245,'임대보증금']=1898000
train.loc[245,'임대료']=47210

#19.31
train.loc[246,'임대보증금']=6600000
train.loc[246,'임대료']=259700

#21.19-1
train.loc[247,'임대보증금']=7400000
train.loc[247,'임대료']=291100

#21.19-2
train.loc[248,'임대보증금']=7600000
train.loc[248,'임대료']=298900

#22.95
train.loc[249,'임대보증금']=0
train.loc[249,'임대료']=0

#23.40-1,2,3
train.loc[250:252,'임대보증금']=0
train.loc[250:252,'임대료']=181820

#23.40-4
train.loc[253,'임대보증금']=8200000
train.loc[253,'임대료']=322500

#27.23
train.loc[254,'임대보증금']=3600000
train.loc[254,'임대료']=124700

#31.85
train.loc[255,'임대보증금']=11000000
train.loc[255,'임대료']=432700

#32.29
train.loc[256,'임대보증금']=4200000
train.loc[256,'임대료']=145400

#35.13
train.loc[257,'임대보증금']=4370000
train.loc[257,'임대료']=121600

#36.47
train.loc[258,'임대보증금']=12600000
train.loc[258,'임대료']=495700

#50.08
train.loc[259,'임대보증금']=6400000
train.loc[259,'임대료']=221600

#240.22
train.loc[260,'임대보증금']=20000000
train.loc[260,'임대료']=442500

#완벽

In [55]:
train[train.단지코드 == 'C1859']
#대전판암4
#https://www.kohom.or.kr/moffice/0602002/MM002004.do

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
261,C1859,2424,아파트,대전광역시,영구임대,26.37,1865,1.00,C,5787000.00,79980.00,1.00,4.00,487.00,116.00
262,C1859,2424,아파트,대전광역시,영구임대,26.37,10,1.00,C,5787000.00,79980.00,1.00,4.00,487.00,116.00
263,C1859,2424,아파트,대전광역시,영구임대,26.37,10,1.00,C,5787000.00,79980.00,1.00,4.00,487.00,116.00
264,C1859,2424,아파트,대전광역시,영구임대,31.32,478,1.00,C,6873000.00,94990.00,1.00,4.00,487.00,116.00
265,C1859,2424,아파트,대전광역시,영구임대,52.74,26,1.00,C,11574000.00,159960.00,1.00,4.00,487.00,116.00
266,C1859,2424,상가,대전광역시,임대상가,38.00,1,1.00,D,nan,nan,1.00,4.00,487.00,116.00
267,C1859,2424,상가,대전광역시,임대상가,38.00,1,1.00,D,nan,nan,1.00,4.00,487.00,116.00
268,C1859,2424,상가,대전광역시,임대상가,31.84,1,1.00,D,nan,nan,1.00,4.00,487.00,116.00
269,C1859,2424,상가,대전광역시,임대상가,31.84,1,1.00,D,nan,nan,1.00,4.00,487.00,116.00
270,C1859,2424,상가,대전광역시,임대상가,31.84,1,1.00,D,nan,nan,1.00,4.00,487.00,116.00


In [56]:
train[(train.단지코드 == 'C1859') & (train.공급유형 == '임대상가')]['전용면적'].unique()

array([ 38.  ,  31.84,  37.26,  37.41,  37.49,  37.95,  38.04,  39.33,
        54.51,  54.6 ,  54.61,  75.98, 126.65, 583.4 ])

In [57]:
train[(train.단지코드 == 'C1859') & (train.공급유형 == '임대상가')]['전용면적'].value_counts()

31.84     14
37.41      3
54.51      3
39.33      3
37.95      2
38.00      2
126.65     1
54.60      1
583.40     1
75.98      1
54.61      1
38.04      1
37.49      1
37.26      1
Name: 전용면적, dtype: int64

In [58]:
#38.00(7701-101)
train.loc[266,'임대보증금']=99050000
train.loc[266,'임대료']=0

#38.00(7701-105)
train.loc[267,'임대보증금']=26807000
train.loc[267,'임대료']=380880

#31.84(7702-101)
train.loc[268,'임대보증금']=12530000
train.loc[268,'임대료']=194700

#31.84(7702-102, 103)
train.loc[269:270,'임대보증금']=12280000
train.loc[269:270,'임대료']=190880

#31.84(7702-104)
train.loc[271,'임대보증금']=11837000
train.loc[271,'임대료']=193830

#31.84(7702-105)
train.loc[272,'임대보증금']=12460000
train.loc[272,'임대료']=189750

#31.84(7702-106)
train.loc[273,'임대보증금']=12100000
train.loc[273,'임대료']=192150

#31.84(7702-107)
train.loc[274,'임대보증금']=12220000
train.loc[274,'임대료']=191350

#31.84(7702-201)
train.loc[275,'임대보증금']=4830000
train.loc[275,'임대료']=38940

#31.84(7702-202)
train.loc[276,'임대보증금']=4830000
train.loc[276,'임대료']=71020

#31.84(7702-203)
train.loc[277,'임대보증금']=4830000
train.loc[277,'임대료']=71060

#31.84(7702-204)
train.loc[278,'임대보증금']=4920000
train.loc[278,'임대료']=70460

#31.84(7702-205)
train.loc[279,'임대보증금']=4930000
train.loc[279,'임대료']=70400

#31.84(7702-206)
train.loc[280,'임대보증금']=4588500
train.loc[280,'임대료']=72670

#31.84(7702-207)
train.loc[281,'임대보증금']=4730000
train.loc[281,'임대료']=71730

#37.26(7701-106)
train.loc[282,'임대보증금']=23471000
train.loc[282,'임대료']=359390

#37.41(7701-107, 108, 109)
train.loc[283:285,'임대보증금']=22625000
train.loc[283:285,'임대료']=356100

#37.49(7701-110)
train.loc[286,'임대보증금']=23617000
train.loc[286,'임대료']=350480

#37.95(7701-207)
train.loc[287,'임대보증금']=2371000
train.loc[287,'임대료']=38540

#37.95(7701-208)
train.loc[288,'임대보증금']=8320000
train.loc[288,'임대료']=138610

#38.04(7701-209)
train.loc[289,'임대보증금']=8630000
train.loc[289,'임대료']=137030

#39.33(7701-102)
train.loc[290,'임대보증금']=18672000
train.loc[290,'임대료']=518680

#39.33(7701-103)
train.loc[291,'임대보증금']=88635000
train.loc[291,'임대료']=0

#39.33(7701-104)
train.loc[292,'임대보증금']=26017000
train.loc[292,'임대료']=405870

#54.51(7701-202, 204)
train.loc[293:294,'임대보증금']=12490000
train.loc[293:294,'임대료']=195490

#54.51(7701-203)
train.loc[295,'임대보증금']=3747000
train.loc[295,'임대료']=53270

#54.60(7701-205)
train.loc[296,'임대보증금']=12870000
train.loc[296,'임대료']=193440

#54.61(7701-201)
train.loc[297,'임대보증금']=13050000
train.loc[297,'임대료']=192280

#75.98(7701-206)
train.loc[298,'임대보증금']=58285000
train.loc[298,'임대료']=0

#126.65(7702-1 지하)
train.loc[299,'임대보증금']=10022400
train.loc[299,'임대료']=146590

#583.4(가동 지하)
train.loc[300,'임대보증금']=0
train.loc[300,'임대료']=300000

#완벽

In [59]:
train[train.단지코드 == 'C1350']
#대전관저5 S1블록 LH다온숲아파트
#공공분양이므로 임대보증금과 임대료가 없음 -> 0으로 처리

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
2331,C1350,1401,아파트,대전광역시,공공분양,74.94,317,2.00,D,nan,nan,nan,6.00,1636.00,2315.00
2332,C1350,1401,아파트,대전광역시,공공분양,74.94,137,2.00,D,nan,nan,nan,6.00,1636.00,2315.00
2333,C1350,1401,아파트,대전광역시,공공분양,74.94,22,2.00,D,nan,nan,nan,6.00,1636.00,2315.00
2334,C1350,1401,아파트,대전광역시,공공분양,84.94,164,2.00,D,nan,nan,nan,6.00,1636.00,2315.00
2335,C1350,1401,아파트,대전광역시,공공분양,84.94,19,2.00,D,nan,nan,nan,6.00,1636.00,2315.00
2336,C1350,1401,아파트,대전광역시,공공분양,84.96,26,2.00,D,nan,nan,nan,6.00,1636.00,2315.00
2337,C1350,1401,아파트,대전광역시,공공분양,84.97,26,2.00,D,nan,nan,nan,6.00,1636.00,2315.00
2338,C1350,1401,아파트,대전광역시,공공임대(10년),51.99,106,2.00,A,28013000.00,408600.00,nan,6.00,1636.00,2315.00
2339,C1350,1401,아파트,대전광역시,공공임대(10년),59.91,13,2.00,A,37474000.00,519350.00,nan,6.00,1636.00,2315.00
2340,C1350,1401,아파트,대전광역시,공공임대(10년),59.92,223,2.00,A,37385000.00,516130.00,nan,6.00,1636.00,2315.00


In [60]:
train.loc[2331:2337,'임대보증금']=0
train.loc[2331:2337,'임대료']=0

In [61]:
train[train.단지코드 == 'C1326']
#부산정관7단지
#https://www.myhome.go.kr/hws/portal/sch/selectRentalHouseInfoListView.do
#부산광역시 기장군 정관읍 모전로 41

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
2547,C1326,1934,아파트,부산광역시,국민임대,24.72,472,43.00,H,nan,nan,0.00,4.00,1670.00,1153.00
2548,C1326,1934,아파트,부산광역시,국민임대,24.79,104,43.00,H,nan,nan,0.00,4.00,1670.00,1153.00
2549,C1326,1934,아파트,부산광역시,국민임대,26.83,590,43.00,H,nan,nan,0.00,4.00,1670.00,1153.00
2550,C1326,1934,아파트,부산광역시,국민임대,37.70,464,43.00,H,nan,nan,0.00,4.00,1670.00,1153.00
2551,C1326,1934,아파트,부산광역시,국민임대,46.94,304,43.00,H,nan,nan,0.00,4.00,1670.00,1153.00


In [62]:
#24.72
train.loc[2547,'임대보증금']=7287000
train.loc[2547,'임대료']=140530

#24.79
train.loc[2548,'임대보증금']=7287000
train.loc[2548,'임대료']=140530

#26.83
train.loc[2549,'임대보증금']=7911000
train.loc[2549,'임대료']=147820

#37.70
train.loc[2550,'임대보증금']=15406000
train.loc[2550,'임대료']=206110

#46.94
train.loc[2551,'임대보증금']=24047000
train.loc[2551,'임대료']=269610

In [63]:
train[train.단지코드 == 'C1786']
#춘천거두2 행복주택
#https://www.myhome.go.kr/hws/portal/sch/selectRentalHouseInfoListView.do
#강원도 춘천시 동내면 거두택지길 70

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
2680,C1786,480,아파트,강원도,행복주택,16.91,156,25.00,K,nan,nan,0.00,3.00,338.00,345.00
2681,C1786,480,아파트,강원도,행복주택,26.90,136,25.00,K,nan,nan,0.00,3.00,338.00,345.00
2682,C1786,480,아파트,강원도,행복주택,26.90,72,25.00,K,nan,nan,0.00,3.00,338.00,345.00
2683,C1786,480,아파트,강원도,행복주택,26.90,24,25.00,K,nan,nan,0.00,3.00,338.00,345.00
2684,C1786,480,아파트,강원도,행복주택,36.91,92,25.00,J,30560000.00,145160.00,0.00,3.00,338.00,345.00


In [64]:
#16.91
train.loc[2680,'임대보증금']=15520000
train.loc[2680,'임대료']=73720

#26.90
train.loc[2681:2683,'임대보증금']=22920000
train.loc[2681:2683,'임대료']=108870

#26.90 -> 임대료 오류 발견해서 수정
train.loc[2684,'임대료']=149660

In [65]:
train[train.단지코드 == 'C2186']
#대구연경 LH천년나무 2단지
#https://www.myhome.go.kr/hws/portal/sch/selectRentalHouseInfoListView.do
#대구광역시 동구 연경지묘로 153

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
2906,C2186,924,아파트,대구광역시,국민임대,29.17,238,0.00,H,nan,nan,0.00,8.00,664.00,744.00
2907,C2186,924,아파트,대구광역시,국민임대,29.34,38,0.00,H,nan,nan,0.00,8.00,664.00,744.00
2908,C2186,924,아파트,대구광역시,국민임대,37.43,220,0.00,H,nan,nan,0.00,8.00,664.00,744.00
2909,C2186,924,아파트,대구광역시,국민임대,46.51,188,0.00,H,25981000.00,248630.00,0.00,8.00,664.00,744.00
2910,C2186,924,아파트,대구광역시,영구임대,26.37,219,0.00,C,13126000.00,105870.00,0.00,8.00,664.00,744.00
2911,C2186,924,아파트,대구광역시,영구임대,26.54,13,0.00,C,13164000.00,106190.00,0.00,8.00,664.00,744.00


In [66]:
#29.17
train.loc[2906,'임대보증금']=10847000
train.loc[2906,'임대료']=138600

#29.34
train.loc[2907,'임대보증금']=10847000
train.loc[2907,'임대료']=138600

#37.43
train.loc[2908,'임대보증금']=17338000
train.loc[2908,'임대료']=197500

#46.51 -> 임대료 오류 발견해서 수정
train.loc[2909,'임대료']=248630

In [67]:
#문주 파트

train.loc[88:90,'임대보증금']=15984000
train.loc[88:90,'임대료']=380000

train.loc[91:91,'임대보증금']=14180000
train.loc[91:91,'임대료']=330800

# 101호 103호 105호 등 공고 자료가 없어서 104호 102호 수치로 대체
train.loc[306:308,'임대보증금']=18430000 
train.loc[306:308,'임대료']=428000
train.loc[309:310,'임대보증금']=19560000 
train.loc[309:310,'임대료']=455000
train.loc[311:311,'임대보증금']=13390000 
train.loc[311:311,'임대료']=316000

#전용면적 16
train.loc[319:319,'임대보증금']=4006000 
train.loc[319:319,'임대료']=111300

#전용면적 14.17
train.loc[320:320,'임대보증금']=3614000 
train.loc[320:320,'임대료']=100400

#전용면적 18.90
train.loc[321:321,'임대보증금']=5029000 
train.loc[321:321,'임대료']=139700

#전용면적 20.52
train.loc[322:322,'임대보증금']=12578000 
train.loc[322:322,'임대료']=349400

#전용면적 22.97
train.loc[323:324,'임대보증금']=14790000 
train.loc[323:324,'임대료']=336000

#전용면적 23.40
train.loc[325:328,'임대보증금']=13170000 
train.loc[325:328,'임대료']=314300

#전용면적 25.88

train.loc[329:329,'임대보증금']=6602000 
train.loc[329:329,'임대료']=183400

#전용면적 27.30
train.loc[330:330,'임대보증금']=7074000 
train.loc[330:330,'임대료']=196500

#전용면적 29.62
train.loc[331:331,'임대보증금']=7545000 
train.loc[331:331,'임대료']=209600

#전용면적 30.62
train.loc[332:332,'임대보증금']=19700000 
train.loc[332:332,'임대료']=447600

#전용면적 35.91 자료를 찾지못해서 1층 30.62와 46.80의 평균으로사용
train.loc[333:333,'임대보증금']=(19700000+36150000)/2
train.loc[333:333,'임대료']=(447600+821100)/2

#전용면적 44.63
train.loc[334:334,'임대보증금']=11793000 
train.loc[334:334,'임대료']=327600

#전용면적 46.80
train.loc[335:335,'임대보증금']=36150000 
train.loc[335:335,'임대료']=821100

#전용면적 32.10 (101호~105호)
train.loc[338:340,'임대보증금']=9070000 
train.loc[338:340,'임대료']=215200
train.loc[341:341,'임대보증금']=8320000 
train.loc[341:341,'임대료']=195100
train.loc[342:342,'임대보증금']=8740000 
train.loc[342:342,'임대료']=204800

#전용면적 72.16 지하 1호
train.loc[343:343,'임대보증금']=7570000 
train.loc[343:343,'임대료']=183500

#전용면적 32.10 (101호~105호)
train.loc[358:358,'임대보증금']=9350000 
train.loc[358:358,'임대료']=216700

train.loc[359:359,'임대보증금']=9070000 
train.loc[359:359,'임대료']=210400

train.loc[360:360,'임대보증금']=9350000 
train.loc[360:360,'임대료']=216700


train.loc[361:361,'임대보증금']=9070000 
train.loc[361:361,'임대료']=210400

train.loc[362:362,'임대보증금']=10579000 
train.loc[362:362,'임대료']=293880

#전용면적 32.10 (201호~203호)

train.loc[363:363,'임대보증금']=5131000 
train.loc[363:363,'임대료']=142530

train.loc[364:364,'임대보증금']=4550000 
train.loc[364:364,'임대료']=105600

train.loc[365:365,'임대보증금']=4510000 
train.loc[365:365,'임대료']=104700

#전용면적 63.68 2층 204호
train.loc[366:366,'임대보증금']=10910000 
train.loc[366:366,'임대료']=223000

#전용면적 126.65 지하 1층
train.loc[367:367,'임대보증금']=10156000 
train.loc[367:367,'임대료']=282130

#전용면적 14.10
train.loc[371:371,'임대보증금']=5500000 
train.loc[371:371,'임대료']=79700

#전용면적 18,98
train.loc[372:372,'임대보증금']=2220000 
train.loc[372:372,'임대료']=32190

#전용면적 19.36
train.loc[373:373,'임대보증금']=11720000 
train.loc[373:373,'임대료']=183300

#전용면적 21.46 109호
train.loc[374:374,'임대보증금']=12370000 
train.loc[374:374,'임대료']=193500

#전용면적 21.46 107호
train.loc[375:375,'임대보증금']=5000000 
train.loc[375:375,'임대료']=0

#전용면적 22.83 207호
train.loc[376:376,'임대보증금']=6150000 
train.loc[376:376,'임대료']=97700

#전용면적 23.40 101호~103호
train.loc[377:379,'임대보증금']=12810000 
train.loc[377:379,'임대료']=200500

#전용면적 23.40 104호
train.loc[380:380,'임대보증금']=4611000 
train.loc[380:380,'임대료']=72150

#전용면적 27.55
train.loc[381:381,'임대보증금']=3225000 
train.loc[381:381,'임대료']=46770

#전용면적 29.17
train.loc[382:382,'임대보증금']=11380000 
train.loc[382:382,'임대료']=165000

#전용면적 31.92
train.loc[383:383,'임대보증금']=18400000 
train.loc[383:383,'임대료']=287900

#전용면적 32.60
train.loc[384:384,'임대보증금']=8790000 
train.loc[384:384,'임대료']=139500

#전용면적 36.57
train.loc[385:385,'임대보증금']=11380000 
train.loc[385:385,'임대료']=165000

#전용면적 50.08
train.loc[386:386,'임대보증금']=5502000 
train.loc[386:386,'임대료']=79830

#전용면적 240.22
train.loc[387:387,'임대보증금']=62250000 
train.loc[387:387,'임대료']=1041900

#전용면적 14.10
train.loc[391:391,'임대보증금']=2136000 
train.loc[391:391,'임대료']=89000

#전용면적 18,98
train.loc[392:392,'임대보증금']=4970000 
train.loc[392:392,'임대료']=80000

#전용면적 19.36 자료가없어서 금곡1단지로 값으로 대체
train.loc[393:393,'임대보증금']=11720000 
train.loc[393:393,'임대료']=183300

#전용면적 21.46 109호
train.loc[394:394,'임대보증금']=11370000 
train.loc[394:394,'임대료']=181060

#전용면적 21.46 자료가없어서 금곡1단지 값으로 대체
train.loc[395:395,'임대보증금']=5000000 
train.loc[395:395,'임대료']=0

#전용면적 22.83 
train.loc[396:396,'임대보증금']=5340000 
train.loc[396:396,'임대료']=83400

#전용면적 23.40 101호
train.loc[397:397,'임대보증금']=11780000 
train.loc[397:397,'임대료']=187550

#전용면적 23.40 102~103호 102호 자료가없어서 103호 값으로 대체
train.loc[398:399,'임대보증금']=11780000 
train.loc[398:399,'임대료']=170500

#전용면적 23.40 102~103호 102호 자료가없어서 103호 값으로 대체
train.loc[400:400,'임대보증금']=13700000 
train.loc[400:400,'임대료']=218240

#전용면적 27.55 자료가없어서 금곡 1단지 값으로 대체
train.loc[401:401,'임대보증금']=3225000 
train.loc[401:401,'임대료']=46770

#전용면적 29.17
train.loc[402:402,'임대보증금']=9840000 
train.loc[402:402,'임대료']=144700

#전용면적 31.92
train.loc[403:403,'임대보증금']=7608000 
train.loc[403:403,'임대료']=317000

#전용면적 32.10 금곡2단지 나동 101호~104호 103호 자료만 찾아서 101,102,104호 값이 없어서 103호 값으로 대체
train.loc[404:407,'임대보증금']=12870000 
train.loc[404:407,'임대료']=225170

#전용면적 32.10 금곡2단지 나동 105호~108호 108호 자료만 찾아서 105,106,107호 값이 없어서 108호 값으로 대체
train.loc[408:411,'임대보증금']=11160000 
train.loc[408:411,'임대료']=177400

#전용면적 32.60
train.loc[412:412,'임대보증금']=9790000 
train.loc[412:412,'임대료']=143900

#전용면적 36.57
train.loc[413:413,'임대보증금']=20350000 
train.loc[413:413,'임대료']=294600

#전용면적 50.08
train.loc[414:414,'임대보증금']=13820000 
train.loc[414:414,'임대료']=220300

#전용면적 72.26
train.loc[415:415,'임대보증금']=10230000 
train.loc[415:415,'임대료']=175500

#전용면적 240.22 금곡 1단지 값으로 대체
train.loc[416:416,'임대보증금']=62250000 
train.loc[416:416,'임대료']=1041900

#전용면적 30.00
train.loc[419:419,['임대보증금','임대료']]=[12490000,168700]

#전용면적 21.85
train.loc[420:420,['임대보증금','임대료']]=[13350000,181000]

#전용면적 21.94
train.loc[421:421,['임대보증금','임대료']]=[13400000,181900]

#전용면적 21.98 108~109호,111호
train.loc[422:424,['임대보증금','임대료']]=[14000000,189900]

#전용면적 21.98 110호
train.loc[425:425,['임대보증금','임대료']]=[13300000,180400]

#전용면적 22.24
train.loc[426:426,['임대보증금','임대료']]=[13580000,184300]

#전용면적 23.35
train.loc[427:427,['임대보증금','임대료']]=[26620000,361300]

#전용면적 25.98
train.loc[428:428,['임대보증금','임대료']]=[15870000,215400]

#전용면적 26.53
train.loc[429:429,['임대보증금','임대료']]=[15400000,208900]

#전용면적 28.38
train.loc[430:430,['임대보증금','임대료']]=[9800000,144800]

#전용면적 28.45
train.loc[431:431,['임대보증금','임대료']]=[16510000,224000]

#전용면적 33.39
train.loc[432:432,['임대보증금','임대료']]=[13250000,179000]

#전용면적 33.51 208호 206호 
train.loc[433:434,['임대보증금','임대료']]=[13290000,179500]

#전용면적 33.51 201호
train.loc[435:435,['임대보증금','임대료']]=[13960000,188500]


#전용면적 36.76
train.loc[436:436,['임대보증금','임대료']]=[14580000,197000]

#전용면적 52.50
train.loc[437:437,['임대보증금','임대료']]=[20830000,281400]

#전용면적 401.50
train.loc[438:438,['임대보증금','임대료']]=[130250000,15154400]

#전용면적 38.00 106호
train.loc[445:445,['임대보증금','임대료']]=[17380000,256100]

#전용면적 38.00 110호
train.loc[446:446,['임대보증금','임대료']]=[17390000,273700]

#전용면적 37.26
train.loc[447:447,['임대보증금','임대료']]=[16550000,240300]

#전용면적 37.41 102호
train.loc[448:448,['임대보증금','임대료']]=[16130000,240300]

#전용면적 37.41 103호
train.loc[449:449,['임대보증금','임대료']]=[13570000,220200]

#전용면적 37.41 104호 
train.loc[450:450,['임대보증금','임대료']]=[16130000,240300]


#전용면적 37.49
train.loc[451:451,['임대보증금','임대료']]=[16650000,243800]

#전용면적 37.95 202호
train.loc[452:452,['임대보증금','임대료']]=[8310000,120800]

#전용면적 37.95 203호
train.loc[453:453,['임대보증금','임대료']]=[8310000,116800]

#전용면적 38.04
train.loc[454:454,['임대보증금','임대료']]=[8320000,121000]

#전용면적 39.33 107호
train.loc[455:455,['임대보증금','임대료']]=[16950000,262900]

#전용면적 39.33 108호
train.loc[456:456,['임대보증금','임대료']]=[16950000,254300]

#전용면적 39.33 109호
train.loc[457:457,['임대보증금','임대료']]=[16950000,262900]

#전용면적 54.51 206호
train.loc[458:458,['임대보증금','임대료']]=[11920000,173300]

#전용면적 54.51 207호
train.loc[459:459,['임대보증금','임대료']]=[10270000,160220]

#전용면적 54.51 208호
train.loc[460:460,['임대보증금','임대료']]=[11780000,171400]

#전용면적 54.60 1개
train.loc[461:461,['임대보증금','임대료']]=[11940000,173600]

#전용면적 54..61 1개
train.loc[462:462,['임대보증금','임대료']]=[11950000,173800]

#전용면적 75.98
train.loc[463:463,['임대보증금','임대료']]=[16430000,238900]

#전용면적 583.40
train.loc[464:464,['임대보증금','임대료']]=[51960000,1191600]

In [68]:
#민석

# C2597

train[train.단지코드 == "C2597"]  #부산동삼  lh + 주택공사

train.loc[468:468,["임대보증금","임대료"]] = [4250000,49000]

train.loc[469:470,["임대보증금","임대료"]] = [4550000,46400]

train.loc[471:471,["임대보증금","임대료"]] = [5990000,62000] # 대충 

train.loc[472:473,["임대보증금","임대료"]] = [6290000,68000] # 대충

train.loc[474:475,["임대보증금","임대료"]] = [6790000,65000] # 대충

train.loc[476:477,["임대보증금","임대료"]] = [7990000,73000]

train.loc[478:484,["임대보증금","임대료"]] = [4330000,57900]

train.loc[485:485,["임대보증금","임대료"]] = [4400000,55900]

train.loc[486:490,["임대보증금","임대료"]] = [8390000,78900]

train.loc[491:492,["임대보증금","임대료"]] = [8550000,98700]

train.loc[493:495,["임대보증금","임대료"]] = [8620000,115200]

train.loc[496:496,["임대보증금","임대료"]] = [23900000,305900]

# C2310

train[train.단지코드 == "C2310"] # 부산동삼2 https://www.kohom.or.kr/moffice/0314002/MM002004.do?page=1

train.loc[500:501,"임대보증금"] = 19433000
train.loc[500:501,"임대료"] = 159800

train.loc[502:502,"임대보증금"] = 16760000
train.loc[502:502,"임대료"] = 137800

train.loc[503:503,"임대보증금"] = 16827000
train.loc[503:503,"임대료"] = 138400

train.loc[504:504,"임대보증금"] = 40000000
train.loc[504:504,"임대료"] = 150000

train.loc[505:505,"임대보증금"] = 0
train.loc[505:505,"임대료"] = 0

train.loc[506:506,"임대보증금"] = 0
train.loc[506:506,"임대료"] = 0

train.loc[507:508,"임대보증금"] = 9560000
train.loc[507:508,"임대료"] = 76100

train.loc[509:509,"임대보증금"] = 9582000
train.loc[509:509,"임대료"] = 76100

train.loc[510:512,["임대보증금","임대료"]] = [17692000, 145500]

train.loc[513:513,["임대보증금","임대료"]] = [4128000, 32850]

train.loc[514:515,["임대보증금","임대료"]] = [13761000, 109500]

train.loc[516:516,["임대보증금","임대료"]] = [13783000, 109500]

train.loc[517:517,["임대보증금","임대료"]] = [13761000, 109500]

train.loc[518:518,["임대보증금","임대료"]] = [18648000, 148500]

train.loc[519:519,["임대보증금","임대료"]] = [107424000, 1013600]

# C2132

train[train.단지코드 == "C2132"] # 부산모라 3


train.loc[526:527,["임대보증금","임대료"]] = [3651000, 51700]

train.loc[528:529,["임대보증금","임대료"]] = [18176000, 330700]

train.loc[530:530,["임대보증금","임대료"]] = [7452000, 104600]

train.loc[531:532,["임대보증금","임대료"]] = [4170000, 69300]

train.loc[533:534,["임대보증금","임대료"]] = [9503000, 132500]

train.loc[535:535,["임대보증금","임대료"]] = [9190000, 129200]

train.loc[536:536,["임대보증금","임대료"]] = [9295000, 130300]

train.loc[537:537,["임대보증금","임대료"]] = [9976000, 139100]

train.loc[538:538,["임대보증금","임대료"]] = [12873000, 180800]

train.loc[539:540,["임대보증금","임대료"]] = [5950000, 91200]

train.loc[541:541,["임대보증금","임대료"]] = [6906000, 97700]

train.loc[542:542,["임대보증금","임대료"]] = [13691000, 189400]

train.loc[543:543,["임대보증금","임대료"]] = [14904000, 209500]

train.loc[544:544,["임대보증금","임대료"]] = [13490000, 211800]

train.loc[545:545,["임대보증금","임대료"]] = [8130000, 115000]

train.loc[546:546,["임대보증금","임대료"]] = [8162000, 115400]

train.loc[547:547,["임대보증금","임대료"]] = [8291000, 117400]

train.loc[548:548,["임대보증금","임대료"]] = [8162000, 115400]

train.loc[549:549,["임대보증금","임대료"]] = [18409000, 332500]

train.loc[550:550,["임대보증금","임대료"]] = [18250000, 329500]

train.loc[551:551,["임대보증금","임대료"]] = [17629000, 320800]

train.loc[552:552,["임대보증금","임대료"]] = [17854000, 325100]

train.loc[553:553,["임대보증금","임대료"]] = [17337000, 328700]

train.loc[554:555,["임대보증금","임대료"]] = [9011000, 149800]

train.loc[556:556,["임대보증금","임대료"]] = [9032000, 150100]

train.loc[557:557,["임대보증금","임대료"]] = [19188000, 346500]

train.loc[558:558,["임대보증금","임대료"]] = [19596000, 352200]

train.loc[559:559,["임대보증금","임대료"]] = [18812000, 342200]

train.loc[560:560,["임대보증금","임대료"]] = [18163000, 255300]

train.loc[561:563,["임대보증금","임대료"]] = [13086000, 214800]

train.loc[564:565,["임대보증금","임대료"]] = [13108000, 214700]

train.loc[566:566,["임대보증금","임대료"]] = [18512000, 290600]

train.loc[567:567,["임대보증금","임대료"]] = [81594000, 1246500]

train.loc[568:568,["임대보증금","임대료"]] = [113167000, 1736900]

# C1439

train[train.단지코드 == "C1439"] # 부산반송

train.loc[572:572,["임대보증금","임대료"]] = [3060000, 48400]

train.loc[573:573,["임대보증금","임대료"]] = [1320000, 55000]

train.loc[574:574,["임대보증금","임대료"]] = [744000, 31000]

train.loc[575:575,["임대보증금","임대료"]] = [1670000, 20800]

train.loc[576:576,["임대보증금","임대료"]] = [6170000, 95200]

train.loc[577:577,["임대보증금","임대료"]] = [1530000, 64000]

train.loc[578:578,["임대보증금","임대료"]] = [3740000, 59300] # 18평 

train.loc[580:581,["임대보증금","임대료"]] = [912000, 38000] # 18평 4개중 3개만 이용 한개는 통합

train.loc[582:584,["임대보증금","임대료"]] = [3456000, 144000]

train.loc[585:585,["임대보증금","임대료"]] = [7940000, 122800]

train.loc[586:586,["임대보증금","임대료"]] = [4810000, 76400]

train.loc[587:588,["임대보증금","임대료"]] = [2184000, 91000] 

train.loc[589:589,["임대보증금","임대료"]] = [1944000, 81000] 

train.loc[590:590,["임대보증금","임대료"]] = [7010000, 229000]

train.loc[591:591,["임대보증금","임대료"]] = [2040000, 85000]

train.loc[592:592,["임대보증금","임대료"]] = [9130000, 141200]

train.loc[593:593,["임대보증금","임대료"]] =  [2184000, 91000]

train.loc[594:594,["임대보증금","임대료"]] =  [1900000, 36400] # 26평 4개중 2개는 따로 2개는 통합

train.loc[595:595,["임대보증금","임대료"]] =  [1296000, 54000] # 

train.loc[599:599,["임대보증금","임대료"]] =  [4704000, 196000]

train.loc[600:600,["임대보증금","임대료"]] =  [2664000, 111000]

train.loc[601:601,["임대보증금","임대료"]] =  [3096000, 129000]

train.loc[602:602,["임대보증금","임대료"]] =  [5448000, 227000]

train.loc[603:603,["임대보증금","임대료"]] =  [5496000, 229000]

train.loc[604:604,["임대보증금","임대료"]] =  [3096000, 129000]

train.loc[605:605,["임대보증금","임대료"]] =  [5730000, 97800]

train.loc[606:606,["임대보증금","임대료"]] =  [1656000, 69000]

train.loc[607:608,["임대보증금","임대료"]] =  [2928000, 122000]

train.loc[609:610,["임대보증금","임대료"]] =  [1656000, 69000]

train.loc[612:612,["임대보증금","임대료"]] =  [6648000, 277000]

train.loc[613:613,["임대보증금","임대료"]] =  [3768000, 157000]

train.loc[614:614,["임대보증금","임대료"]] =  [3072000, 128000]

train.loc[615:615,["임대보증금","임대료"]] =  [13440000, 560000]

train.loc[616:616,["임대보증금","임대료"]] =  [21168000, 882000]

#579, 596, 597, 598, 611 ==> 합쳐서 132.77 통합건물 보증금 5천 임대료 18

train.loc[579:579,["전용면적", "임대보증금","임대료"]] = [132.77,50000000,180000]

train = train.drop( [596,597,598,611],axis = 0)

# C1899

train[train.단지코드 == "C1899"]

train.loc[624:624,["임대보증금","임대료"]] =  [874000, 204800]

train.loc[625:625,["임대보증금","임대료"]] =  [832000, 195100]

train.loc[626:627,["임대보증금","임대료"]] =  [907000, 215200]

# C1056

train[train.단지코드 == "C1056"]

train.loc[630:634,["임대보증금","임대료"]] =  [425000, 69750]

train.loc[635:639,["임대보증금","임대료"]] =  [724000, 122460] #2개는 0인데 똑같은걸로채우

# C2644

train[train.단지코드 == "C2644"]

train.loc[644:644,["임대보증금","임대료"]] =  [13610000, 317700]

train.loc[645:646,["임대보증금","임대료"]] =  [16240000, 451117]

# C1206

train[train.단지코드 == "C1206"]

train.loc[649:653,["임대보증금","임대료"]] =  [13180000, 311000]

train.loc[654:656,["임대보증금","임대료"]] =  [6120000, 151000]

train.loc[657:657,["임대보증금","임대료"]] =  [13180000, 311000] # 이거

train.loc[658:658,["임대보증금","임대료"]] =  [26360000, 622000] # 이건 안나와서 2배 4배해줌

In [69]:
#정하
# C1775
# 제주아라
# https://www.kohom.or.kr/moffice/1604001/MM002004.do
# 제주시 안다12길1 제주아라아파트


# 31.84 _ 1층상가
train.loc[674:680,["임대보증금","임대료"]] = [9166000,229800]

# 31.84 _ 2층상가
train.loc[681:687,["임대보증금","임대료"]] = [5053000, 138960]

# 137.49
train.loc[688,["임대보증금","임대료"]] = [0, 3525000]


# C1790
# 진주가좌1
# https://www.kohom.or.kr/moffice/1313001/MM002004.do
# 경상남도 진주시 가호로 26 ( 가좌동, 진주가좌주공아파트 )


#15.00 
train.loc[702:702,"임대보증금"] = 5690000
train.loc[702:702,"임대료"] = 136000

#19.15
train.loc[703:703,"임대보증금"] = 4120000
train.loc[703:703,"임대료"] = 99000

#19.25
train.loc[704:705,"임대보증금"] = 6670000
train.loc[704:705,"임대료"] = 1595000

#22.89
train.loc[706:706,"임대보증금"] = 11780000
train.loc[706:706,"임대료"] = 253300

# 23.25
train.loc[707:707,"임대보증금"] = 5010000
train.loc[707:707,"임대료"] = 120200

#23.88
train.loc[708:708,"임대보증금"] = 9050000
train.loc[708:708,"임대료"] = 216300

# 23.91
train.loc[709:709,"임대보증금"] = 9540000
train.loc[709:709,"임대료"] = 227900

#24.38
train.loc[710:711,"임대보증금"] = 9230000
train.loc[710:711,"임대료"] = 220700

#27.50
train.loc[712:713,"임대보증금"] = 9540000
train.loc[712:713,"임대료"] = 228000

#27.75
train.loc[714:721,"임대보증금"] = 5980000
train.loc[714:721,"임대료"] = 143500

#28.19
train.loc[722:722,"임대보증금"] = 6080000
train.loc[722:722,"임대료"] = 146000

#28.88
train.loc[723:727,"임대보증금"] = 11530000
train.loc[723:727,"임대료"] = 275600

#42.35
train.loc[728:728,"임대보증금"] = 9590000
train.loc[728:728,"임대료"] = 230200

#55.17
train.loc[729:729,"임대보증금"] = 12490000
train.loc[729:729,"임대료"] = 299900

#82.92
train.loc[730:730,"임대보증금"] = 21480000
train.loc[730:730,"임대료"] = 453300

#248.56
train.loc[731:731,"임대보증금"] = 32000000
train.loc[731:731,"임대료"] = 785000


# C2109
# 진주평거1
# 경상남도 진주시 진양흐로 125번길 


# 31.84
train.loc[734:747,"임대보증금"] = 3369000
train.loc[734:747,"임대료"] = 93600

# 126.65
train.loc[748,"임대보증금"] = 8334000
train.loc[748,"임대료"] = 231500



# C1698
# 진해자은
# https://www.kohom.or.kr/moffice/1314001/MM002004.do
# 경상남도 창원시 진해군 진해대로 901번지 11


# 32.10 / 101호, 104호
train.loc[753:754,"임대보증금"] = 6620000
train.loc[753:754,"임대료"] = 157500

# 32.10 / 102호
train.loc[755,["임대보증금","임대료"]] = [5920000, 140900]

# 32.10 / 103호
train.loc[756,["임대보증금","임대료"]] = [5920000, 156600]


# C2571
# 통영도남
# https://www.kohom.or.kr/moffice/1317001/MM002004.do?page=1
# 경상남도 통영시 발개로153 

# 31.84 _ 101호
train.loc[826,["임대보증금","임대료"]] = [4860000, 111000]

# 31.84 _ 102호
train.loc[827,["임대보증금","임대료"]] = [5130000, 94930]

# 31.84 _ 103~105호
train.loc[828:830,["임대보증금","임대료"]] = [4590000, 113100]

# 31.84 _ 106호
train.loc[831,["임대보증금","임대료"]] = [5040000, 124300]

# 31.84 _ 201호
train.loc[832,["임대보증금","임대료"]] = [2670000, 62000]

# 31.84 _ 202,206호
train.loc[833:834,["임대보증금","임대료"]] = [2820000, 65400]

# 31.84 _ 203호
train.loc[835,["임대보증금","임대료"]] = [2970000, 68900]

# 31.84 _ 204호
train.loc[836,["임대보증금","임대료"]] = [3100000, 71900]

# 31.84 _ 205호
train.loc[837,["임대보증금","임대료"]] = [2960000, 68600]

# 126.65 _ 지하1호
train.loc[838,["임대보증금","임대료"]] = [7900000, 0]


# C2212
# 춘천석사3 
# https://www.kohom.or.kr/moffice/1513002/MM002004.do?page=1
# 강원도 춘천시 후석로 228번길 47

# 31.84 _ 1층
train.loc[811:813,["임대보증금","임대료"]] = [13560000, 334000]

# 31.84 _ 2층
train.loc[814:816,["임대보증금","임대료"]] = [5120000, 132000]

# 63.68 _ 2층
train.loc[817:819,["임대보증금","임대료"]] = [10240000, 265000]

# 63.68 _ 1층
train.loc[820,["임대보증금","임대료"]] = [26910000, 664000]

# 63.68 _ 1층
train.loc[821,["임대보증금","임대료"]] = [27150000, 670000]

# 137.49
train.loc[822,["임대보증금","임대료"]] = [14594900, 361200]



# C1875 
# 천안쌍용1
# https://www.kohom.or.kr/moffice/1112003/MM002004.do?page=1
# 충청남도 천안시 서북구 쌍용동 주공7단지아파트



# 14.10
train.loc[786,["임대보증금","임대료"]] = [2619000, 63450]

# 18.98
train.loc[787,["임대보증금","임대료"]] = [5172000, 143670]

# 19.36
train.loc[788,["임대보증금","임대료"]] = [9120000, 222600]

# 21.46
train.loc[789:790,["임대보증금","임대료"]] = [10010000, 244200]

# 22.83
train.loc[791,["임대보증금","임대료"]] = [5380000, 130000]

# 23.40 _ 102~105
train.loc[792:795,["임대보증금","임대료"]] = [11140000, 271600]

# 23.40 _ 101
train.loc[796,["임대보증금","임대료"]] = [11250000, 274300]

# 23.40 _ 106
train.loc[797,["임대보증금","임대료"]] = [15480000, 274300]

# 27.55
train.loc[798,["임대보증금","임대료"]] = [5280000, 127600]

# 29.17
train.loc[799,["임대보증금","임대료"]] = [6641000, 184490]

# 31.92
train.loc[800,["임대보증금","임대료"]] = [14820000, 361500]

# 32.60
train.loc[801,["임대보증금","임대료"]] = [7758000, 215510]

# 36.57
train.loc[802,["임대보증금","임대료"]] = [16730000, 407900]

# 50.08
train.loc[803,["임대보증금","임대료"]] = [13166000, 365720]

# 240.22
train.loc[804,["임대보증금","임대료"]] = [35520000, 869400]



# C1004
# 천안성정4
# https://www.kohom.or.kr/moffice/1112001/MM002004.do?page=1
# 천안성정4(주공6단지)아파트

# 19.00
train.loc[766,["임대보증금","임대료"]] = [3200000, 72460]

# 14.10
train.loc[767,["임대보증금","임대료"]] = [2600000, 42490]

# 19.31
train.loc[768,["임대보증금","임대료"]] = [4680000, 204080]

# 21.19
train.loc[769:770,["임대보증금","임대료"]] = [8890000, 156360]

# 22.95
train.loc[771,["임대보증금","임대료"]] = [0, 0]

# 23.40 _ 101 
train.loc[772,["임대보증금","임대료"]] = [8890000, 156360]

# 23.40 _ 102,103 
train.loc[773:774,["임대보증금","임대료"]] = [6400000, 253700]

# 23.40 _ 104
train.loc[775,["임대보증금","임대료"]] = [11900000, 172180]

# 27.23
train.loc[776,["임대보증금","임대료"]] = [3860000, 61990]

# 31.85
train.loc[777,["임대보증금","임대료"]] = [11950000, 210130]

# 32.29
train.loc[778,["임대보증금","임대료"]] = [4590000, 73540]

# 35.13
train.loc[779,["임대보증금","임대료"]] = [5250000, 90050]

# 36.47
train.loc[780,["임대보증금","임대료"]] = [11007000, 193440]

# 50.08
train.loc[781,["임대보증금","임대료"]] = [8400000, 190130]

# 240.22
train.loc[782,["임대보증금","임대료"]] = [20000000, 638720]

## 임대상가의 임대보증금, 임대료 결측치 처리 [test]
- 'C1006', 'C2676', 'C2177', 'C2033', 'C1812', 'C2253', 'C1729', 'C2152', 'C1267'

In [70]:
test[test.임대보증금.isna()].단지코드.unique()

array(['C1006', 'C2676', 'C2177', 'C2033', 'C1812', 'C2253', 'C1729',
       'C2152', 'C1267'], dtype=object)

In [71]:
test[test.임대료.isna()].단지코드.unique()

array(['C1006', 'C2676', 'C2177', 'C2033', 'C1812', 'C2253', 'C1729',
       'C2152', 'C1267'], dtype=object)

In [72]:
test[test.단지코드=='C1006']
# 대전둔산1
# https://www.kohom.or.kr/moffice/0603002/MM002004.do?page=1
# 대전광역시 서구 월평북로 11

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
73,C1006,1505,아파트,대전광역시,영구임대,26.37,358,27.00,C,5787000.00,79980.00,2.00,5.00,428.00
74,C1006,1505,아파트,대전광역시,영구임대,26.37,229,27.00,C,5787000.00,79980.00,2.00,5.00,428.00
75,C1006,1505,아파트,대전광역시,영구임대,26.37,574,27.00,C,5787000.00,79980.00,2.00,5.00,428.00
76,C1006,1505,아파트,대전광역시,영구임대,26.37,10,27.00,D,nan,nan,2.00,5.00,428.00
77,C1006,1505,아파트,대전광역시,영구임대,26.37,10,27.00,D,nan,nan,2.00,5.00,428.00
78,C1006,1505,아파트,대전광역시,영구임대,31.32,298,27.00,C,6873000.00,94990.00,2.00,5.00,428.00
79,C1006,1505,아파트,대전광역시,영구임대,52.74,6,27.00,D,nan,nan,2.00,5.00,428.00
80,C1006,1505,상가,대전광역시,임대상가,38.00,1,27.00,D,nan,nan,2.00,5.00,428.00
81,C1006,1505,상가,대전광역시,임대상가,38.00,1,27.00,D,nan,nan,2.00,5.00,428.00
82,C1006,1505,상가,대전광역시,임대상가,37.26,1,27.00,D,nan,nan,2.00,5.00,428.00


In [73]:
test[(test.단지코드 == 'C1006') & (test.공급유형 == '임대상가')]['전용면적'].unique()

array([ 38.  ,  37.26,  37.41,  37.49,  37.95,  38.04,  39.33,  39.93,
        54.51,  54.6 ,  54.61,  75.98, 583.4 ])

In [74]:
test[(test.단지코드 == 'C1006') & (test.공급유형 == '임대상가')]['전용면적'].value_counts()

54.51     3
37.41     3
37.95     2
39.93     2
38.00     2
39.33     1
583.40    1
54.60     1
38.04     1
75.98     1
54.61     1
37.49     1
37.26     1
Name: 전용면적, dtype: int64

In [75]:
# 아파트 결측값

# 26.37
test.loc[76:77,['임대보증금','임대료']]=[2500000,49760]

# 52.74
test.loc[79,['임대보증금','임대료']]=[5000000,99520]

# 상가 결측값

# 38.00
test.loc[80,['임대보증금','임대료']]=[29450000,838310]

# 38.00
test.loc[81,['임대보증금','임대료']]=[28730000,826210]

# 37.26
test.loc[82,['임대보증금','임대료']]=[24930000, 709720]

# 37.41
test.loc[83:85,['임대보증금','임대료']]=[23170000, 659670]

# 37.49
test.loc[86,['임대보증금','임대료']]=[25130000, 715440]

# 37.95
test.loc[87,['임대보증금','임대료']]=[27700000, 142120]

# 37.95
test.loc[88,['임대보증금','임대료']]=[9700000, 274120]

# 38.04
test.loc[89,['임대보증금','임대료']]=[9820000, 277420]

# 39.93
test.loc[90:92,['임대보증금','임대료']]=[25010000, 712030]

# 54.51
test.loc[93:95,['임대보증금','임대료']]=[14560000, 411290]

# 54.60
test.loc[96,['임대보증금','임대료']]=[14730000, 416240]

# 54.61
test.loc[97,['임대보증금','임대료']]=[14730000, 416240]

# 75.98
test.loc[98,['임대보증금','임대료']]=[23280000, 671990]

# 583.4
test.loc[99,['임대보증금','임대료']]=[151120000, 4022370]

In [76]:
test[test.단지코드=='C2676']
# 부산모라1
# https://www.kohom.or.kr/moffice/0309001/MM002004.do?page=1
# 부산광역시 사상구 모라동 1366-2

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
113,C2676,2572,아파트,부산광역시,영구임대,26.37,294,25.00,C,5787000.00,79980.00,0.00,2.00,418.00
114,C2676,2572,아파트,부산광역시,영구임대,26.37,1341,25.00,C,5787000.00,79980.00,0.00,2.00,418.00
115,C2676,2572,아파트,부산광역시,영구임대,31.32,894,25.00,C,6873000.00,94990.00,0.00,2.00,418.00
116,C2676,2572,상가,부산광역시,임대상가,15.00,1,25.00,D,nan,nan,0.00,2.00,418.00
117,C2676,2572,상가,부산광역시,임대상가,15.00,1,25.00,D,nan,nan,0.00,2.00,418.00
118,C2676,2572,상가,부산광역시,임대상가,38.00,1,25.00,D,nan,nan,0.00,2.00,418.00
119,C2676,2572,상가,부산광역시,임대상가,38.00,1,25.00,D,nan,nan,0.00,2.00,418.00
120,C2676,2572,상가,부산광역시,임대상가,16.57,1,25.00,D,nan,nan,0.00,2.00,418.00
121,C2676,2572,상가,부산광역시,임대상가,18.38,1,25.00,D,nan,nan,0.00,2.00,418.00
122,C2676,2572,상가,부산광역시,임대상가,18.38,1,25.00,D,nan,nan,0.00,2.00,418.00


In [77]:
test[(test.단지코드 == 'C2676') & (test.공급유형 == '임대상가')]['전용면적'].unique()

array([ 15.  ,  38.  ,  16.57,  18.38,  20.9 ,  21.94,  23.35,  26.25,
        28.38,  28.5 ,  33.15,  33.31,  33.39,  33.51,  37.26,  37.41,
        37.49,  37.95,  38.04,  39.33,  40.39,  54.51,  54.6 ,  54.61,
        75.98, 404.65, 583.4 ])

In [78]:
test[(test.단지코드 == 'C2676') & (test.공급유형 == '임대상가')]['전용면적'].value_counts()

20.90     4
39.33     3
37.41     3
33.51     3
54.51     3
38.00     2
18.38     2
26.25     2
15.00     2
37.95     2
16.57     1
33.15     1
583.40    1
40.39     1
28.50     1
38.04     1
37.26     1
33.39     1
54.60     1
23.35     1
404.65    1
28.38     1
33.31     1
75.98     1
54.61     1
37.49     1
21.94     1
Name: 전용면적, dtype: int64

In [79]:
# 15.00
test.loc[116 ,['임대보증금','임대료']]=[3070000 ,49700 ]

# 15.00
test.loc[117 ,['임대보증금','임대료']]=[3070000 ,497000 ]

# 38.00
test.loc[118 ,['임대보증금','임대료']]=[8736000 ,364000 ]

# 38.00
test.loc[119,['임대보증금','임대료']]=[ 16720000,273500 ]

# 16.57
test.loc[120,['임대보증금','임대료']]=[6870000 ,106300 ]

# 18.38
test.loc[121,['임대보증금','임대료']]=[1824000 ,76000 ]

# 18.38
test.loc[122 ,['임대보증금','임대료']]=[3760000 ,61000 ]

# 20.90
test.loc[123 ,['임대보증금','임대료']]=[4344000 ,181000 ]

# 20.90
test.loc[124:125 ,['임대보증금','임대료']]=[ 3912000, 163000]

# 20.90
test.loc[126 ,['임대보증금','임대료']]=[9080000 ,140500 ]

# 21.94
test.loc[127 ,['임대보증금','임대료']]=[4104000 ,171000 ]

# 23.35
test.loc[128 ,['임대보증금','임대료']]=[10740000 ,166200 ]

# 26.25
test.loc[129 ,['임대보증금','임대료']]=[2616000 ,109000 ]

# 26.25
test.loc[130 ,['임대보증금','임대료']]=[2616000 ,109000 ]

# 28.38
test.loc[131 ,['임대보증금','임대료']]=[5810000 ,94200 ]

# 28.50
test.loc[132 ,['임대보증금','임대료']]=[5328000 ,222000 ]

# 33.15
test.loc[133 ,['임대보증금','임대료']]=[13480000 ,208700 ]

# 33.31
test.loc[134 ,['임대보증금','임대료']]=[6240000 ,260000 ]

# 33.39
test.loc[135 ,['임대보증금','임대료']]=[3336000 ,139000 ]

# 33.51
test.loc[136 ,['임대보증금','임대료']]=[3336000 ,139000 ]

# 33.51
test.loc[137 ,['임대보증금','임대료']]=[6860000 ,111300 ]

# 33.51
test.loc[138 ,['임대보증금','임대료']]=[6860000 ,111300 ]

# 37.26
test.loc[139 ,['임대보증금','임대료']]=[14900000 ,243800 ]

# 37.41
test.loc[140 ,['임대보증금','임대료']]=[14960000 ,244800 ]

# 37.41
test.loc[141 ,['임대보증금','임대료']]=[35000000 ,96630 ]

# 37.41
test.loc[142 ,['임대보증금','임대료']]=[12870000 ,225800 ]

# 37.49
test.loc[143 ,['임대보증금','임대료']]=[7344000 ,306000 ]

# 37.95
test.loc[144 ,['임대보증금','임대료']]=[8440000 ,138000 ]

# 37.95
test.loc[145 ,['임대보증금','임대료']]=[8440000 ,136100 ]

# 38.04
test.loc[146 ,['임대보증금','임대료']]=[8460000 ,132900 ]

# 39.33
test.loc[147:148 ,['임대보증금','임대료']]=[ 17300000,283100 ]

# 39.33
test.loc[149 ,['임대보증금','임대료']]=[17300000 ,272100 ]

# 40.39
test.loc[150 ,['임대보증금','임대료']]=[15950000 ,247000 ]

# 54.51
test.loc[151:153 ,['임대보증금','임대료']]=[12120000 ,198200 ]

# 54.60
test.loc[154 ,['임대보증금','임대료']]=[12140000 ,198500 ]

# 54.61
test.loc[155 ,['임대보증금','임대료']]=[12150000 ,198600 ]

# 75.98
test.loc[156 ,['임대보증금','임대료']]=[17470000 ,285600 ]

# 404.65
test.loc[157 ,['임대보증금','임대료']]=[31824000 ,1326000 ]

# 583.40
test.loc[158 ,['임대보증금','임대료']]=[70000000 , 0]

In [80]:
test[test.단지코드=='C2177']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
168,C2177,1245,아파트,충청남도,영구임대,26.37,478,2.00,C,3141000.00,69900.00,nan,1.00,187.00
169,C2177,1245,아파트,충청남도,영구임대,26.37,596,2.00,C,3141000.00,69900.00,nan,1.00,187.00
170,C2177,1245,아파트,충청남도,영구임대,31.32,149,2.00,C,3731000.00,83020.00,nan,1.00,187.00
171,C2177,1245,상가,충청남도,임대상가,15.00,1,2.00,D,nan,nan,nan,1.00,187.00
172,C2177,1245,상가,충청남도,임대상가,15.00,1,2.00,D,nan,nan,nan,1.00,187.00
173,C2177,1245,상가,충청남도,임대상가,16.57,1,2.00,D,nan,nan,nan,1.00,187.00
174,C2177,1245,상가,충청남도,임대상가,18.38,1,2.00,D,nan,nan,nan,1.00,187.00
175,C2177,1245,상가,충청남도,임대상가,18.38,1,2.00,D,nan,nan,nan,1.00,187.00
176,C2177,1245,상가,충청남도,임대상가,20.90,1,2.00,D,nan,nan,nan,1.00,187.00
177,C2177,1245,상가,충청남도,임대상가,20.90,1,2.00,D,nan,nan,nan,1.00,187.00


In [81]:
test[(test.단지코드 == 'C2177') & (test.공급유형 == '임대상가')]['전용면적'].unique()

array([ 15.  ,  16.57,  18.38,  20.9 ,  21.94,  23.35,  26.25,  28.5 ,
        33.15,  33.31,  33.39,  33.51,  40.39,  61.89, 401.5 ])

In [82]:
test[(test.단지코드 == 'C2177') & (test.공급유형 == '임대상가')]['전용면적'].value_counts()

20.90     4
33.51     2
18.38     2
26.25     2
15.00     2
33.31     1
23.35     1
33.39     1
40.39     1
33.15     1
61.89     1
16.57     1
21.94     1
401.50    1
28.50     1
Name: 전용면적, dtype: int64

In [83]:
test.loc[171:171,["임대보증금","임대료"]] =  [2380000, 59200]
test.loc[172:172,["임대보증금","임대료"]] =  [2390000, 59500]

test.loc[173:173,["임대보증금","임대료"]] =  [5140000, 128000]

test.loc[174:174,["임대보증금","임대료"]] =  [2850000, 70900]

test.loc[175:175,["임대보증금","임대료"]] =  [2565000, 63740]

test.loc[176:177,["임대보증금","임대료"]] =  [6360000, 158200]

test.loc[178:178,["임대보증금","임대료"]] =  [6200000, 154300]

test.loc[179:179,["임대보증금","임대료"]] =  [6280000, 156200]

test.loc[180:180,["임대보증금","임대료"]] =  [6574000, 163590]

test.loc[181:181,["임대보증금","임대료"]] =  [10200000, 238000]

test.loc[182:182,["임대보증금","임대료"]] =  [4000000, 99500]

test.loc[183:183,["임대보증금","임대료"]] =  [3591000, 89360]

test.loc[184:184,["임대보증금","임대료"]] =  [8189000, 223630]

test.loc[185:185,["임대보증금","임대료"]] =  [10250000, 255000]

test.loc[186:186,["임대보증금","임대료"]] =  [10530000, 261900]

test.loc[187:187,["전용면적","임대보증금","임대료"]] = [33.51, 4653000,115920]

test.loc[188:188,["임대보증금","임대료"]] =  [4617000, 114830]

test.loc[189:189,["임대보증금","임대료"]] =  [4815000, 119700]

test.loc[190:190,["임대보증금","임대료"]] =  [12220000, 243000]

test.loc[191:191,["임대보증금","임대료"]] =  [9324000, 217530]

test.loc[192:192,["임대보증금","임대료"]] =  [50341000, 1201400]

In [84]:
test[test.단지코드=='C2033']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
198,C2033,1400,아파트,울산광역시,영구임대,26.37,573,12.00,C,4032000.00,74860.00,0.00,7.00,308.00
199,C2033,1400,아파트,울산광역시,영구임대,26.37,358,12.00,C,4032000.00,74860.00,0.00,7.00,308.00
200,C2033,1400,아파트,울산광역시,영구임대,26.37,447,12.00,C,4032000.00,74860.00,0.00,7.00,308.00
201,C2033,1400,상가,울산광역시,임대상가,15.00,1,12.00,D,nan,nan,0.00,7.00,308.00
202,C2033,1400,상가,울산광역시,임대상가,15.00,1,12.00,D,nan,nan,0.00,7.00,308.00
203,C2033,1400,상가,울산광역시,임대상가,16.57,1,12.00,D,nan,nan,0.00,7.00,308.00
204,C2033,1400,상가,울산광역시,임대상가,18.38,1,12.00,D,nan,nan,0.00,7.00,308.00
205,C2033,1400,상가,울산광역시,임대상가,18.38,1,12.00,D,nan,nan,0.00,7.00,308.00
206,C2033,1400,상가,울산광역시,임대상가,20.90,1,12.00,D,nan,nan,0.00,7.00,308.00
207,C2033,1400,상가,울산광역시,임대상가,20.90,1,12.00,D,nan,nan,0.00,7.00,308.00


In [85]:
test[(test.단지코드 == 'C2033') & (test.공급유형 == '임대상가')]['전용면적'].unique()

array([ 15.  ,  16.57,  18.38,  20.9 ,  21.94,  23.35,  26.25,  28.5 ,
        33.15,  33.31,  33.39,  33.51,  40.39,  61.89, 401.5 ])

In [86]:
test[(test.단지코드 == 'C2033') & (test.공급유형 == '임대상가')]['전용면적'].value_counts()

20.90     4
33.51     2
18.38     2
26.25     2
15.00     2
33.31     1
23.35     1
33.39     1
40.39     1
33.15     1
61.89     1
16.57     1
21.94     1
401.50    1
28.50     1
Name: 전용면적, dtype: int64

In [87]:
test.loc[201:202,["임대보증금","임대료"]] =  [2810000, 66700]

test.loc[203:203,["임대보증금","임대료"]] =  [5150000, 123000]

test.loc[204:204,["임대보증금","임대료"]] =  [3450000, 81700]

test.loc[205:205,["임대보증금","임대료"]] =  [3270000, 77700]

test.loc[206:209,["임대보증금","임대료"]] =  [7040000, 168000]

test.loc[210:210,["임대보증금","임대료"]] =  [7190000, 171600]

test.loc[211:211,["임대보증금","임대료"]] =  [11680000, 260200]

test.loc[212:213,["임대보증금","임대료"]] =  [4920000, 116800]

test.loc[214:214,["임대보증금","임대료"]] =  [8870000, 211700]

test.loc[215:215,["임대보증금","임대료"]] =  [10870000, 259300]

test.loc[216:216,["임대보증금","임대료"]] =  [11470000, 273600]

test.loc[217:217,["임대보증금","임대료"]] =  [6260000, 148600]

test.loc[218:219,["임대보증금","임대료"]] =  [6290000, 149200]

test.loc[220:220,["임대보증금","임대료"]] =  [12840000, 306500]

test.loc[221:221,["임대보증금","임대료"]] =  [11470000, 283590]

test.loc[222:222,["임대보증금","임대료"]] =  [62650000, 1387700]

In [88]:
test[test.단지코드=='C1812']
#울산화정
#총세대수보다 전용면적별세대수 합계가 13작음
#https://www.kohom.or.kr/moffice/0802001/MM002004.do

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
231,C1812,1021,아파트,울산광역시,영구임대,26.37,745,22.00,C,3922000.00,72830.00,0.00,4.00,185.00
232,C1812,1021,아파트,울산광역시,영구임대,31.32,239,22.00,C,4659000.00,86500.00,0.00,4.00,185.00
233,C1812,1021,상가,울산광역시,임대상가,9.96,1,22.00,D,nan,nan,0.00,4.00,185.00
234,C1812,1021,상가,울산광역시,임대상가,13.07,1,22.00,D,nan,nan,0.00,4.00,185.00
235,C1812,1021,상가,울산광역시,임대상가,13.59,1,22.00,D,nan,nan,0.00,4.00,185.00
236,C1812,1021,상가,울산광역시,임대상가,14.25,1,22.00,D,nan,nan,0.00,4.00,185.00
237,C1812,1021,상가,울산광역시,임대상가,14.25,1,22.00,D,nan,nan,0.00,4.00,185.00
238,C1812,1021,상가,울산광역시,임대상가,14.25,1,22.00,D,nan,nan,0.00,4.00,185.00
239,C1812,1021,상가,울산광역시,임대상가,14.25,1,22.00,D,nan,nan,0.00,4.00,185.00
240,C1812,1021,상가,울산광역시,임대상가,19.15,1,22.00,D,nan,nan,0.00,4.00,185.00


In [89]:
test[(test.단지코드 == 'C1812') & (test.공급유형 == '임대상가')]['전용면적'].unique()

array([  9.96,  13.07,  13.59,  14.25,  19.15,  22.89,  23.25,  27.75,
        28.19,  42.35,  55.17, 248.56, 253.71])

In [90]:
test[(test.단지코드 == 'C1812') & (test.공급유형 == '임대상가')]['전용면적'].value_counts()

27.75     9
14.25     4
55.17     2
19.15     1
22.89     1
13.07     1
9.96      1
42.35     1
248.56    1
28.19     1
13.59     1
253.71    1
23.25     1
Name: 전용면적, dtype: int64

In [91]:
#오류 수정
test.loc[231,["임대보증금","임대료"]] = [2304000, 46000]
test.loc[232,["임대보증금","임대료"]] = [2737000, 54000]

#9.96(113호)
test.loc[233,["임대보증금","임대료"]] = [1250000, 39000]

#13.07(114호)
test.loc[234,["임대보증금","임대료"]] = [1910000, 59400]

#13.59(112호)
test.loc[235,["임대보증금","임대료"]] = [1710000, 53200]

#14.25(108~111호)
test.loc[236:239,["임대보증금","임대료"]] = [1810000, 55900]

#19.15(120호)
test.loc[240,["임대보증금","임대료"]] = [4460000, 107400]

#22.89(121호)
test.loc[241,["임대보증금","임대료"]] = [3910000, 150400]

#23.25(213호)
test.loc[242,["임대보증금","임대료"]] = [1860000, 56600]

#27.75(205호, 211호)
test.loc[243:244,["임대보증금","임대료"]] = [2220000, 67600]

#28.19

#42.35
test.loc[253,["임대보증금","임대료"]] = [3560000, 108500]

#55.17

#248.56

#253.71

In [92]:
#없는 것은 찾은 것들의 평균으로 대치
test.loc[test.단지코드.isin(['C1812']) & test.임대보증금.isnull(), '임대보증금'] = test.loc[test.단지코드.isin(['C1812']) & (test.공급유형 == '임대상가')]['임대보증금'].mean()
test.loc[test.단지코드.isin(['C1812']) & test.임대료.isnull(), '임대료'] = test.loc[test.단지코드.isin(['C1812']) & (test.공급유형 == '임대상가')]['임대료'].mean()

In [93]:
test[test.단지코드=='C2253']
#춘천명륜2

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.00,C,2249000.00,44770.00,0.00,2.00,173.00
259,C2253,1161,아파트,강원도,영구임대,31.32,239,0.00,C,3731000.00,83020.00,0.00,2.00,173.00
260,C2253,1161,아파트,강원도,영구임대,31.32,149,0.00,C,3731000.00,83020.00,0.00,2.00,173.00
261,C2253,1161,상가,강원도,임대상가,13.77,1,0.00,D,nan,nan,0.00,2.00,173.00
262,C2253,1161,상가,강원도,임대상가,22.89,1,0.00,D,nan,nan,0.00,2.00,173.00
263,C2253,1161,상가,강원도,임대상가,22.91,1,0.00,D,nan,nan,0.00,2.00,173.00
264,C2253,1161,상가,강원도,임대상가,23.79,1,0.00,D,nan,nan,0.00,2.00,173.00
265,C2253,1161,상가,강원도,임대상가,23.79,1,0.00,D,nan,nan,0.00,2.00,173.00
266,C2253,1161,상가,강원도,임대상가,23.86,1,0.00,D,nan,nan,0.00,2.00,173.00
267,C2253,1161,상가,강원도,임대상가,23.86,1,0.00,D,nan,nan,0.00,2.00,173.00


In [94]:
test[(test.단지코드 == 'C2253') & (test.공급유형 == '임대상가')]['전용면적'].unique()

array([ 13.77,  22.89,  22.91,  23.79,  23.86,  24.19,  27.75,  28.19,
        28.69,  28.93,  42.35,  42.4 ,  55.17,  55.5 ,  82.92, 248.56])

In [95]:
test[(test.단지코드 == 'C2253') & (test.공급유형 == '임대상가')]['전용면적'].value_counts()

27.75     6
28.69     5
23.79     2
23.86     2
28.93     2
82.92     1
42.40     1
13.77     1
22.91     1
55.17     1
22.89     1
42.35     1
248.56    1
28.19     1
55.50     1
24.19     1
Name: 전용면적, dtype: int64

In [96]:
test.loc[261,["임대보증금","임대료"]] =  [5260000, 135000]

test.loc[262,["임대보증금","임대료"]] =  [9270000, 239000]

test.loc[263,["임대보증금","임대료","전용면적"]] =  [8340000, 215000, 28.69] # 로 추측됨

test.loc[264,["임대보증금","임대료"]] =  [5264000, 135800]

test.loc[265,["임대보증금","임대료"]] =  [7520000, 194000]

test.loc[266,["임대보증금","임대료"]] =  [10180000, 263000]

test.loc[267,["임대보증금","임대료"]] =  [8680000, 224000]

test.loc[268,["임대보증금","임대료"]] =  [8810000, 227000]

test.loc[269:274,["임대보증금","임대료"]] =  [4280000,108000]

test.loc[275,["임대보증금","임대료"]] =  [4360000, 110000]

test.loc[276:280,["임대보증금","임대료"]] = [10450000,270000] #데이터에 28.69평 하나없음 8340000 215000

test.loc[281:282,["임대보증금","임대료"]] = [9160000,236000]


test.loc[283,["임대보증금","임대료"]] = [6540000,166000]

test.loc[284,["임대보증금","임대료"]] = [6860000,174000]

test.loc[285,["임대보증금","임대료"]] = [8520000,216000]

#test.loc[286:287,["임대보증금","임대료"]] =  없음

test.loc[288,["임대보증금","임대료"]] = [13920000,580000]

In [97]:
#없는 것은 찾은 것들의 평균으로 대치
test.loc[test.단지코드.isin(['C2253']) & test.임대보증금.isnull(), '임대보증금'] = test.loc[test.단지코드.isin(['C2253']) & (test.공급유형 == '임대상가')]['임대보증금'].mean()
test.loc[test.단지코드.isin(['C2253']) & test.임대료.isnull(), '임대료'] = test.loc[test.단지코드.isin(['C2253']) & (test.공급유형 == '임대상가')]['임대료'].mean()

In [98]:
test[test.단지코드=='C1729']
#춘천효자8

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
308,C1729,627,아파트,강원도,영구임대,32.76,180,0.00,C,5009000.00,93000.00,0.00,5.00,131.00
309,C1729,627,아파트,강원도,영구임대,32.76,64,0.00,C,5009000.00,93000.00,0.00,5.00,131.00
310,C1729,627,아파트,강원도,영구임대,32.76,27,0.00,C,5009000.00,93000.00,0.00,5.00,131.00
311,C1729,627,아파트,강원도,영구임대,32.76,44,0.00,C,5009000.00,93000.00,0.00,5.00,131.00
312,C1729,627,아파트,강원도,영구임대,42.48,180,0.00,C,6496000.00,120600.00,0.00,5.00,131.00
313,C1729,627,아파트,강원도,영구임대,42.48,56,0.00,C,6496000.00,120600.00,0.00,5.00,131.00
314,C1729,627,아파트,강원도,영구임대,42.48,23,0.00,C,6496000.00,120600.00,0.00,5.00,131.00
315,C1729,627,아파트,강원도,영구임대,42.48,36,0.00,C,6496000.00,120600.00,0.00,5.00,131.00
316,C1729,627,상가,강원도,임대상가,13.02,1,0.00,D,nan,nan,0.00,5.00,131.00
317,C1729,627,상가,강원도,임대상가,18.54,1,0.00,D,nan,nan,0.00,5.00,131.00


In [99]:
test[(test.단지코드 == 'C1729') & (test.공급유형 == '임대상가')]['전용면적'].unique()

array([ 13.02,  18.54,  19.08,  21.86,  22.28,  23.4 ,  27.57,  32.21,
        32.46,  34.86,  35.76,  50.08, 240.22])

In [100]:
test[(test.단지코드 == 'C1729') & (test.공급유형 == '임대상가')]['전용면적'].value_counts()

23.40     4
21.86     2
22.28     1
50.08     1
34.86     1
18.54     1
13.02     1
240.22    1
32.21     1
19.08     1
27.57     1
32.46     1
35.76     1
Name: 전용면적, dtype: int64

In [101]:
#13.02

#18.54

#19.08

#21.86

#22.28

#23.40

#27.57

#32.21

#32.46

#34.86

#35.76

#50.08

#240.22

In [102]:
test[test.단지코드=='C2152']
#영구임대 결측치
#화천신읍
#https://www.myhome.go.kr/hws/portal/sch/selectRsdtRcritNtcDetailView.do?pblancId=9070

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
1005,C2152,120,아파트,강원도,영구임대,24.83,66,9.00,C,nan,nan,0.00,1.00,40.00
1006,C2152,120,아파트,강원도,영구임대,33.84,54,9.00,C,nan,nan,0.00,1.00,40.00


In [103]:
test[(test.단지코드 == 'C2152') & (test.공급유형 == '영구임대')]['전용면적'].unique()

array([24.83, 33.84])

In [104]:
#24.83
test.loc[1005,'임대보증금']=2129000
test.loc[1005,'임대료']=42350

#33.84
test.loc[1006,'임대보증금']=2902000
test.loc[1006,'임대료']=57730

In [105]:
test[test.단지코드=='C1267']
#행복주택 결측치
#창원가포A
#https://www.myhome.go.kr/hws/portal/sch/selectRsdtRcritNtcDetailView.do?pblancId=6373

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수
1007,C1267,675,아파트,경상남도,국민임대,24.87,28,38.00,H,6882000.00,104370.00,0.00,1.00,467.00
1008,C1267,675,아파트,경상남도,국민임대,24.99,90,38.00,H,6882000.00,104370.00,0.00,1.00,467.00
1009,C1267,675,아파트,경상남도,국민임대,24.99,12,38.00,H,6882000.00,104370.00,0.00,1.00,467.00
1010,C1267,675,아파트,경상남도,국민임대,46.72,68,38.00,H,24420000.00,219230.00,0.00,1.00,467.00
1011,C1267,675,아파트,경상남도,국민임대,46.72,112,38.00,H,24420000.00,219230.00,0.00,1.00,467.00
1012,C1267,675,아파트,경상남도,영구임대,22.86,96,38.00,C,10876000.00,89230.00,0.00,1.00,467.00
1013,C1267,675,아파트,경상남도,영구임대,22.86,14,38.00,C,10876000.00,89230.00,0.00,1.00,467.00
1014,C1267,675,아파트,경상남도,행복주택,16.94,50,38.00,L,nan,nan,0.00,1.00,467.00
1015,C1267,675,아파트,경상남도,행복주택,26.85,66,38.00,L,nan,nan,0.00,1.00,467.00
1016,C1267,675,아파트,경상남도,행복주택,26.85,8,38.00,L,nan,nan,0.00,1.00,467.00


In [106]:
test[(test.단지코드 == 'C1267') & (test.공급유형 == '행복주택')]['전용면적'].unique()

array([16.94, 26.85, 36.77])

In [107]:
#16.94
test.loc[1014,'임대보증금']= (30*10880000 + 20*11520000)/50
test.loc[1014,'임대료']= (30*51680 + 20*54720)/50

#26.85-1
test.loc[1015,'임대보증금']= (27*16660000 + 27*17640000 + 12*14700000)/66
test.loc[1015,'임대료']= (27*79130 + 27*83790 + 12+69820)/66

#26.85-2
test.loc[1016,'임대보증금']=18620000
test.loc[1016,'임대료']=88440

#36.77
test.loc[1017,'임대보증금']= (36*23760000 + 68*26400000 + 4*19800000 + 18*25080000)/126
test.loc[1017,'임대료']= (36*112860 + 68*125400 + 4*94050 + 18*119130)/126

##### 나머지 전처리

In [108]:
# 임대보증금과 임대료가 NULL인 경우는 0으로 대체하는 것이 좋아보임
train[['임대보증금', '임대료']] = train[['임대보증금', '임대료']].fillna(0)
test[['임대보증금', '임대료']] = test[['임대보증금', '임대료']].fillna(0)

In [109]:
# 지하철과 버스가 NULL인 경우는 0으로 대체하는 것이 좋아보임
train['지하철'] = train['지하철'].fillna(0)
test['지하철'] = test['지하철'].fillna(0)

train['버스'] = train['버스'].fillna(0)
test['버스'] = test['버스'].fillna(0)

## 중복 확인

In [110]:
train.shape, train.drop_duplicates().shape 

((2865, 15), (2667, 15))

In [111]:
test.shape, test.drop_duplicates().shape

((1022, 14), (971, 14))

In [112]:
# 중복제거
train = train.drop_duplicates()
test = test.drop_duplicates()

## 단지코드 별 집계

##### (1) 하나의 단지코드에 하나의 값만 존재하는 변수들
- '총세대수', '지역', '공가수', '지하철', '버스', '단지내주차면수', '등록차량수'

In [113]:
unique_cols = ['총세대수', '지역', '공가수', '지하철', '버스', '단지내주차면수', '등록차량수']
train_agg = train.set_index('단지코드')[unique_cols].drop_duplicates()
test_agg = test.set_index('단지코드')[[col for col in unique_cols if col!='등록차량수']].drop_duplicates()

In [114]:
train_agg

,총세대수,지역,공가수,지하철,버스,단지내주차면수,등록차량수
단지코드,,,,,,,
C2515,545,경상남도,17.00,0.00,3.00,624.00,205.00
C1407,1216,대전광역시,13.00,1.00,1.00,1285.00,1064.00
C1945,755,경기도,6.00,1.00,3.00,734.00,730.00
C1470,696,전라북도,14.00,0.00,2.00,645.00,553.00
C1898,566,전라북도,9.00,0.00,6.00,517.00,415.00
...,...,...,...,...,...,...,...
C2586,90,제주특별자치도,7.00,0.00,3.00,66.00,57.00
C2035,492,강원도,24.00,0.00,1.00,521.00,246.00
C2020,40,부산광역시,7.00,1.00,2.00,25.00,19.00


##### (2) 하나의 단지코드에 둘 이상의 값이 존재하는 변수들
- 임대건물구분, 공급유형, 전용면적, 전용면적별세대수, 자격유형, 임대보증금, 임대료

##### [임대건물구분]: 아파트/상가

In [115]:
def reshape_cat_features(data, cast_col, value_col):
    res = data.drop_duplicates(['단지코드', cast_col]).assign(counter=1).pivot(index='단지코드', columns=cast_col, values=value_col).fillna(0)
    res.columns.name = None
    res = res.rename(columns={col:cast_col+'_'+col for col in res.columns})
    return res

In [116]:
reshape_cat_features(data=train, cast_col='임대건물구분', value_col='counter')

,임대건물구분_상가,임대건물구분_아파트
단지코드,,
C1000,0.00,1.00
C1004,1.00,1.00
C1005,0.00,1.00
C1013,0.00,1.00
C1014,0.00,1.00
...,...,...
C2663,0.00,1.00
C2666,0.00,1.00
C2670,0.00,1.00


In [117]:
reshape_cat_features(data=test, cast_col='임대건물구분', value_col='counter')

,임대건물구분_상가,임대건물구분_아파트
단지코드,,
C1003,0.00,1.00
C1006,1.00,1.00
C1016,0.00,1.00
C1019,0.00,1.00
C1030,0.00,1.00
...,...,...
C2653,0.00,1.00
C2675,0.00,1.00
C2676,1.00,1.00


##### [공급유형]
- 국민임대와 공공임대의 차이
LH공사와 SH공사에서 제공하는 국민임대는 국가 재정과 국민주택기금을 지원받아 국가, 지방자치단체, 한국토지주택공사 또는 지방공사가 건설, 공급하는 주택을 의미한다. 
저렴한 임대료를 지불하고 장기(30년) 임대가 가능하다. 장기 거주는 가능하지만 해당 주거지를 매입할 수 없다는 단점이 있다.
반면 공공임대는 5년 또는 10년 후 임대기간이 종료되었을 때 입주자에게 우선 분양전환혜택을 준다. 

In [118]:
pd.concat([train.공급유형.value_counts(), test.공급유형.value_counts()], axis=1)

,공급유형,공급유형
국민임대,1699,619.00
임대상가,403,134.00
행복주택,192,121.00
공공임대(10년),173,34.00
영구임대,149,44.00
공공임대(50년),31,13.00
공공임대(분납),7,6.00
공공분양,7,nan
장기전세,3,nan
공공임대(5년),3,nan


In [119]:
train.loc[train.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
test.loc[test.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
train.loc[train.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'
test.loc[test.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'

In [120]:
set(train.공급유형)

{'공공임대(50년)', '공공임대(5년/10년/분납/분양)', '국민임대/장기전세', '영구임대', '임대상가', '행복주택'}

In [121]:
set(test.공급유형)

{'공공임대(50년)', '공공임대(5년/10년/분납/분양)', '국민임대/장기전세', '영구임대', '임대상가', '행복주택'}

In [122]:
reshape_cat_features(data=train, cast_col='공급유형', value_col='counter')

,공급유형_공공임대(50년),공급유형_공공임대(5년/10년/분납/분양),공급유형_국민임대/장기전세,공급유형_영구임대,공급유형_임대상가,공급유형_행복주택
단지코드,,,,,,
C1000,0.00,0.00,1.00,0.00,0.00,0.00
C1004,0.00,0.00,0.00,1.00,1.00,0.00
C1005,0.00,0.00,1.00,0.00,0.00,0.00
C1013,0.00,0.00,1.00,0.00,0.00,0.00
C1014,0.00,0.00,1.00,0.00,0.00,0.00
...,...,...,...,...,...,...
C2663,0.00,0.00,1.00,0.00,0.00,0.00
C2666,0.00,0.00,1.00,0.00,0.00,0.00
C2670,0.00,0.00,1.00,0.00,0.00,0.00


##### [자격유형]

- 변수생성 컨셉: 특정자격 유형이 많으면 그 단지는 등록차량수가 적을 것이다.
예를들어, 소득수준이 낮은 자격유형의 세대가 많으면 주차수요가 적을 가능성이 높을 것으로 예상해볼 수 있다.
문제는 특성이 비슷한 항목끼리 묶어야 의미가 있을 것 같은데, 비식별화 되어 있어서 각 코드값이 무엇인지 정확히 알 수 없다는 것임
단지코드별로 자격유형별 세대수를 알 수 있으면 좋을 것 같지만 이 데이터에서는 파악 불가능
자격유형별 소득수준을 간접적으로 파악하기 위해 자격유형별 임대보증금의 평균, 임대료의 평균을 변수로 사용해 볼 수 있을 것 같음


- 영구임대: 생계급여 또는 의료급여 수급자 등[소득 1분위] - 1
- 국민임대: 무주택세대구성원[소득 2~4분위] - 3
- 장기전세: 무주택세대구성원[소득 3~4분위] - 2
- 공공임대(5년/10년/분납): 무주택세대구성원[소득 3~5분위] - 3
- 행복주택: 무주택세대구성원/무주택자[소득 2~5분위] - 4
- 이렇게 나누면 13개 항목인데 자격유형의 항목수는 15개

In [123]:
pd.concat([train.자격유형.value_counts(), 
           test.자격유형.value_counts()], axis=1)
# B, F, O가 train에만 있고 test에는 없음

,자격유형,자격유형
A,1703,569.00
D,410,136.00
H,154,92.00
J,94,81.00
C,92,35.00
I,49,7.00
E,37,10.00
L,33,12.00
K,33,16.00
N,29,10.00


In [124]:
train.loc[train.자격유형=='B', ['임대건물구분', '공급유형']].drop_duplicates()
# 자격유형 B는 공급유형이 모두 국민임대/장기전세, 공급유형이 국민임대/장기전세 일때 자격유형이 어떤게 있는지 보는게 좋을 듯

,임대건물구분,공급유형
26,아파트,국민임대/장기전세


##### 국민임대/장기전세

- (일반)해당지역 거주 무주택세대구성원
- (특별/우선)3자녀 이상 가구
- (특별/우선)국가유공자
- (특별/우선)영구임대 입주자
- (특별/우선)비닐간이공작물 거주자
- (특별/우선)신혼부부(혼인기간 5년이내)
- (특별/우선)사업지구 철거민
- (특별/우선)기타 공급대상(고령자, 노부모부양자, 장애인, 파독근로자 등)
- 건수가 가장 많은 A는 해당지역 거주 무주택세대구성원으로 예상됨

In [125]:
train.loc[train.공급유형.isin(['국민임대/장기전세']), '자격유형'].value_counts()

A    1487
H     154
E      34
B      18
G       9
Name: 자격유형, dtype: int64

##### 영구임대

- (일반)생계급여 또는 의료급여 수급자
- (일반)국가유공자
- (특별/우선)수급자 선정기준의 소득인정액 이하인 국가유공자
- (특별/우선)귀환국군포로
- (특별/우선)수급자 신혼부부

In [126]:
train.loc[train.공급유형.isin(['영구임대']), '자격유형'].value_counts()

C    92
I    49
F     3
E     3
A     2
Name: 자격유형, dtype: int64

##### 공공임대(5년/10년/분납/분양)

- (일반)해당지역 거주무주택세대구성원
- (특별/우선)다자녀 특별
- (특별/우선)신혼부부 특별
- (특별/우선)생애최초 특별
- (특별/우선)노부모부양 특별
- (특별/우선)국가유공자 특별
- (특별/우선)기타 특별(장애인, 철거민 등)
- A는 '(일반)해당지역 거주무주택세대구성원'으로 추정됨

In [127]:
train.loc[train.공급유형.isin(['공공임대(5년/10년/분납/분양)']), '자격유형'].value_counts()

A    183
D      7
Name: 자격유형, dtype: int64

##### 행복주택

- 대학생(취준생 포함)
- 사회초년생(제취준생 포함)
- 신혼부부(예비신혼부부, 대학생, 취준생 신혼부부 포함)
- 고령자
- 주거급여수급자
- 산업단지 근로자

In [128]:
train.loc[train.공급유형.isin(['행복주택']), '자격유형'].value_counts()

J    94
K    33
L    33
N    29
M     2
O     1
Name: 자격유형, dtype: int64

In [129]:
train.loc[train.자격유형.isin(['J', 'L', 'K', 'N', 'M', 'O']), '공급유형'].value_counts()
# 'J', 'L', 'K', 'N', 'M', 'O' 는 공급유형이 행복주택인 경우에서만 나타남 이것만 따로 묶는게 좋을듯

행복주택    192
Name: 공급유형, dtype: int64

In [130]:
train.loc[train.자격유형.isin(['J', 'L', 'K', 'N', 'M', 'O']), '자격유형'] = '행복주택_공급대상'
test.loc[test.자격유형.isin(['J', 'L', 'K', 'N', 'M', 'O']), '자격유형'] = '행복주택_공급대상'

In [131]:
train.loc[train.자격유형.isin(['A']), '공급유형'].value_counts()

국민임대/장기전세             1487
공공임대(5년/10년/분납/분양)     183
공공임대(50년)               31
영구임대                     2
Name: 공급유형, dtype: int64

In [132]:
train.loc[train.자격유형.isin(['D']), '공급유형'].value_counts()

임대상가                  403
공공임대(5년/10년/분납/분양)      7
Name: 공급유형, dtype: int64

In [133]:
train.loc[train.자격유형.isin(['H']), '공급유형'].value_counts()

국민임대/장기전세    154
Name: 공급유형, dtype: int64

In [134]:
train.loc[train.자격유형.isin(['C']), '공급유형'].value_counts()

영구임대    92
Name: 공급유형, dtype: int64

In [135]:
train.loc[train.자격유형.isin(['I']), '공급유형'].value_counts()

영구임대    49
Name: 공급유형, dtype: int64

In [136]:
train.loc[train.자격유형.isin(['E']), '공급유형'].value_counts()

국민임대/장기전세    34
영구임대          3
Name: 공급유형, dtype: int64

In [137]:
train.loc[train.자격유형.isin(['B']), '공급유형'].value_counts()

국민임대/장기전세    18
Name: 공급유형, dtype: int64

In [138]:
train.loc[train.자격유형.isin(['G']), '공급유형'].value_counts()

국민임대/장기전세    9
Name: 공급유형, dtype: int64

In [139]:
train.loc[train.자격유형.isin(['F']), '공급유형'].value_counts()

영구임대    3
Name: 공급유형, dtype: int64

- 'H', 'B', 'E', 'G' 는 국민임대/장기전세 공급대상(E는 영구임대인 경우도 있긴 하지만 국민임대/장기전세인 경우가 더 많아서 여기에 포함시킴)
- 'C', 'I', 'F', 'G' 는 영구임대 공급대상으로 묶어 보겠음

In [140]:
train.loc[train.자격유형.isin(['H', 'B', 'E', 'G']), '자격유형'] = '국민임대/장기전세_공급대상'
test.loc[test.자격유형.isin(['H', 'B', 'E', 'G']), '자격유형'] = '국민임대/장기전세_공급대상'

train.loc[train.자격유형.isin(['C', 'I', 'F']), '자격유형'] = '영구임대_공급대상'
test.loc[test.자격유형.isin(['C', 'I', 'F']), '자격유형'] = '영구임대_공급대상'

In [141]:
pd.concat([train.자격유형.value_counts(), 
           test.자격유형.value_counts()], axis=1)

,자격유형,자격유형
A,1703,569
D,410,136
국민임대/장기전세_공급대상,218,103
행복주택_공급대상,192,121
영구임대_공급대상,144,42


In [142]:
reshape_cat_features(data=train, cast_col='자격유형', value_col='counter')

,자격유형_A,자격유형_D,자격유형_국민임대/장기전세_공급대상,자격유형_영구임대_공급대상,자격유형_행복주택_공급대상
단지코드,,,,,
C1000,1.00,0.00,0.00,0.00,0.00
C1004,0.00,1.00,0.00,1.00,0.00
C1005,1.00,0.00,0.00,0.00,0.00
C1013,1.00,0.00,0.00,0.00,0.00
C1014,1.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...
C2663,0.00,0.00,1.00,0.00,0.00
C2666,1.00,0.00,0.00,0.00,0.00
C2670,1.00,0.00,0.00,0.00,0.00


In [143]:
reshape_cat_features(data=test, cast_col='자격유형', value_col='counter')

,자격유형_A,자격유형_D,자격유형_국민임대/장기전세_공급대상,자격유형_영구임대_공급대상,자격유형_행복주택_공급대상
단지코드,,,,,
C1003,0.00,0.00,0.00,0.00,1.00
C1006,0.00,1.00,0.00,1.00,0.00
C1016,1.00,0.00,0.00,0.00,0.00
C1019,1.00,0.00,0.00,0.00,0.00
C1030,0.00,0.00,0.00,0.00,1.00
...,...,...,...,...,...
C2653,1.00,0.00,0.00,0.00,0.00
C2675,1.00,0.00,0.00,0.00,0.00
C2676,0.00,1.00,0.00,1.00,0.00


In [144]:
train_agg = pd.concat([train_agg,
                       reshape_cat_features(data=train, cast_col='임대건물구분', value_col='counter'),
                       reshape_cat_features(data=train, cast_col='공급유형', value_col='counter'),
                       reshape_cat_features(data=train, cast_col='자격유형', value_col='counter')], axis=1)

test_agg = pd.concat([test_agg,
                       reshape_cat_features(data=test, cast_col='임대건물구분', value_col='counter'),
                       reshape_cat_features(data=test, cast_col='공급유형', value_col='counter'),
                       reshape_cat_features(data=test, cast_col='자격유형', value_col='counter')], axis=1)

In [145]:
train_agg = train_agg.reset_index()
train_agg.rename(columns={'index':'단지코드'}, inplace= True)
train_agg.head()

,단지코드,총세대수,지역,공가수,지하철,버스,단지내주차면수,등록차량수,임대건물구분_상가,임대건물구분_아파트,...,공급유형_공공임대(5년/10년/분납/분양),공급유형_국민임대/장기전세,공급유형_영구임대,공급유형_임대상가,공급유형_행복주택,자격유형_A,자격유형_D,자격유형_국민임대/장기전세_공급대상,자격유형_영구임대_공급대상,자격유형_행복주택_공급대상
0,C2515,545,경상남도,17.00,0.00,3.00,624.00,205.00,0.00,1.00,...,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
1,C1407,1216,대전광역시,13.00,1.00,1.00,1285.00,1064.00,0.00,1.00,...,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
2,C1945,755,경기도,6.00,1.00,3.00,734.00,730.00,0.00,1.00,...,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
3,C1470,696,전라북도,14.00,0.00,2.00,645.00,553.00,0.00,1.00,...,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
4,C1898,566,전라북도,9.00,0.00,6.00,517.00,415.00,0.00,1.00,...,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00


In [146]:
test_agg = test_agg.reset_index()
test_agg.rename(columns={'index':'단지코드'}, inplace= True)
test_agg.head()

,단지코드,총세대수,지역,공가수,지하철,버스,단지내주차면수,임대건물구분_상가,임대건물구분_아파트,공급유형_공공임대(50년),공급유형_공공임대(5년/10년/분납/분양),공급유형_국민임대/장기전세,공급유형_영구임대,공급유형_임대상가,공급유형_행복주택,자격유형_A,자격유형_D,자격유형_국민임대/장기전세_공급대상,자격유형_영구임대_공급대상,자격유형_행복주택_공급대상
0,C1072,754,경기도,14.00,0.00,2.00,683.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00
1,C1128,1354,경기도,9.00,0.00,3.00,1216.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00
2,C1456,619,부산광역시,18.00,0.00,16.00,547.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
3,C1840,593,전라북도,7.00,0.00,3.00,543.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
4,C1332,1297,경기도,11.00,0.00,2.00,1112.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00


In [147]:
# 미리 밑에 피쳐들과의 중복제거
train_drop_col = ['총세대수', '공가수', '지역', '단지내주차면수', '지하철', '버스', '등록차량수']
test_drop_col = ['총세대수', '공가수', '지역', '단지내주차면수', '지하철', '버스']

train_agg.drop(train_drop_col, axis = 1, inplace = True)
test_agg.drop(test_drop_col, axis=1, inplace = True)

In [148]:
train_agg.shape, test_agg.shape

((411, 14), (150, 14))

## Feature

##### 지역명 숫자로 매핑

In [149]:
#local_map = {}
#for i, loc in enumerate(train['지역'].unique()):
#    local_map[loc] = i
    
#train['지역'] = train['지역'].map(local_map)
#test['지역'] = test['지역'].map(local_map)
#age_gender['지역'] = age_gender['지역'].map(local_map)

##### 전용면적을 3의 배수로 변경

In [150]:
#train.전용면적.sort_values().unique()

In [151]:
train['전용면적'] = train['전용면적']//3*3
test['전용면적'] = test['전용면적']//3*3

In [152]:
train['전용면적'].unique()

array([ 33.,  39.,  45.,  51.,  30.,  57.,  36.,  42.,  48.,  24.,  72.,
        12.,  15.,  21.,  27.,  54.,  18., 315.,  63., 135.,  75., 108.,
       582., 126., 240., 399., 246., 405., 132.,  60., 402.,  81.,  84.,
        66.])

##### 단지별 데이터 1차원으로 취합

In [153]:
columns = ['단지코드', '총세대수', '공가수', '지역', '단지내주차면수', '지하철', '버스']
target = '등록차량수'
area_columns = []
for area in train['전용면적'].unique():
    area_columns.append(f'면적_{area}')

In [154]:
#area_columns

In [155]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()

In [156]:
from tqdm import tqdm

for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()

411it [00:09, 41.73it/s]
150it [00:03, 42.77it/s]


In [157]:
new_train

,단지코드,총세대수,공가수,지역,단지내주차면수,지하철,버스,면적_33.0,면적_39.0,면적_45.0,...,면적_399.0,면적_246.0,면적_405.0,면적_132.0,면적_60.0,면적_402.0,면적_81.0,면적_84.0,면적_66.0,등록차량수
0,C2515,545.00,17.00,경상남도,624.00,0.00,3.00,276.00,80.00,57.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,205.00
1,C1407,1216.00,13.00,대전광역시,1285.00,1.00,1.00,0.00,292.00,170.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1064.00
2,C1945,755.00,6.00,경기도,734.00,1.00,3.00,0.00,120.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,730.00
3,C1470,696.00,14.00,전라북도,645.00,0.00,2.00,0.00,0.00,196.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,553.00
4,C1898,566.00,9.00,전라북도,517.00,0.00,6.00,0.00,0.00,209.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,415.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,C2586,90.00,7.00,제주특별자치도,66.00,0.00,3.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,57.00
407,C2035,492.00,24.00,강원도,521.00,0.00,1.00,0.00,0.00,180.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,246.00
408,C2020,40.00,7.00,부산광역시,25.00,1.00,2.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,19.00
409,C2437,90.00,12.00,충청북도,30.00,0.00,1.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,16.00


In [158]:
# pca
#train_pca = new_train.iloc[:,7:-1]
#test_pca = new_test.iloc[:,7:]

#new_train = new_train.drop(new_train.iloc[:,7:-1].columns, axis=1)
#new_test = new_test.drop(new_test.iloc[:,7:].columns, axis=1)

### Numeric

In [159]:
features = []
features_te = []

In [160]:
numeric_columns = train.dtypes[train.dtypes != 'object'].index.tolist()
train[numeric_columns].head()

,총세대수,전용면적,전용면적별세대수,공가수,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
8,545,33.00,276,17.00,9216000.00,82940.00,0.00,3.00,624.00,205.00
9,545,39.00,60,17.00,12672000.00,107130.00,0.00,3.00,624.00,205.00
10,545,39.00,20,17.00,12672000.00,107130.00,0.00,3.00,624.00,205.00
11,545,45.00,38,17.00,18433000.00,149760.00,0.00,3.00,624.00,205.00
12,545,45.00,19,17.00,18433000.00,149760.00,0.00,3.00,624.00,205.00


In [161]:
#agg_dict = {
    # 기존 컬럼
    #'총세대수':['sum'],
    #'지역':['sum', 'std'],
    #'전용면적':['sum', 'std'],
    #'전용면적별세대수':['min', 'max', 'sum', 'std'],
    #'공가수':['sum', 'std'],
    #'임대보증금':['min', 'max', 'mean', 'sum', 'std'],
    #'임대료':['min', 'max', 'mean', 'sum', 'std'],
    #'지하철':['sum', 'std'],
    #'버스':['sum', 'std'],
    #'단지내주차면수':['sum', 'std']
#}

In [162]:
#f = train.groupby('단지코드').agg(agg_dict)
#f = f.fillna(0)
#f.columns = [('_').join(column).upper() for column in f.columns.ravel()]

#f = f.reset_index()
#features.append(f); f

In [163]:
#f_te = test.groupby('단지코드').agg(agg_dict)
#f_te = f_te.fillna(0)
#f_te.columns = [('_').join(column).upper() for column in f_te.columns.ravel()]

#f_te = f_te.reset_index()
#features_te.append(f); f_te

##### [feature1]: 전용면적당 임대보증금

In [164]:
f = (train.groupby('단지코드')['임대보증금'].agg([sum]) / train.groupby('단지코드')['전용면적'].agg([sum])).reset_index()
f = f.rename(columns = {'sum':'deposit_ratio'})
f = f.fillna(0)
features.append(f); f

,단지코드,deposit_ratio
0,C1000,299110.03
1,C1004,179299.26
2,C1005,322786.16
3,C1013,553884.44
4,C1014,502276.28
...,...,...
406,C2663,766042.19
407,C2666,449773.81
408,C2670,406716.42
409,C2680,562055.56


In [165]:
f_te = (test.groupby('단지코드')['임대보증금'].agg([sum]) / test.groupby('단지코드')['전용면적'].agg([sum])).reset_index()
f_te = f_te.rename(columns = {'sum':'deposit_ratio'})
f_te = f_te.fillna(0)
features_te.append(f_te); f_te

,단지코드,deposit_ratio
0,C1003,745679.01
1,C1006,351544.79
2,C1016,516704.30
3,C1019,341551.72
4,C1030,2056533.33
...,...,...
145,C2653,447842.86
146,C2675,761774.10
147,C2676,208720.86
148,C2688,293494.25


##### [feature2]: 전용면적당 임대료

In [166]:
f = (train.groupby('단지코드')['임대료'].agg([sum]) / train.groupby('단지코드')['전용면적'].agg([sum])).reset_index()
f = f.rename(columns = {'sum':'rent_ratio'})
f = f.fillna(0)
features.append(f); f

,단지코드,rent_ratio
0,C1000,2742.56
1,C1004,4037.07
2,C1005,4189.56
3,C1013,2993.91
4,C1014,3373.63
...,...,...
406,C2663,6811.90
407,C2666,3170.12
408,C2670,2865.37
409,C2680,3090.87


In [167]:
f_te = (test.groupby('단지코드')['임대료'].agg([sum]) / test.groupby('단지코드')['전용면적'].agg([sum])).reset_index()
f_te = f_te.rename(columns = {'sum':'rent_ratio'})
f_te = f_te.fillna(0)
features_te.append(f_te); f_te

,단지코드,rent_ratio
0,C1003,3814.81
1,C1006,9023.06
2,C1016,4903.44
3,C1019,2906.44
4,C1030,8568.67
...,...,...
145,C2653,4629.62
146,C2675,5144.88
147,C2676,3788.93
148,C2688,3767.30


##### [feature3]: 임대보증금 대비 임대료 비율

In [168]:
f = (train.groupby('단지코드')['임대료'].agg([sum]) / train.groupby('단지코드')['임대보증금'].agg([sum])).reset_index()
f = f.rename(columns = {'sum':'rent_per_depoist_ratio'})
f = f.fillna(0)
features.append(f); f

,단지코드,rent_per_depoist_ratio
0,C1000,0.01
1,C1004,0.02
2,C1005,0.01
3,C1013,0.01
4,C1014,0.01
...,...,...
406,C2663,0.01
407,C2666,0.01
408,C2670,0.01
409,C2680,0.01


In [169]:
f_te = (test.groupby('단지코드')['임대료'].agg([sum]) / test.groupby('단지코드')['임대보증금'].agg([sum])).reset_index()
f_te = f_te.rename(columns = {'sum':'rent_per_depoist_ratio'})
f_te = f_te.fillna(0)
features_te.append(f_te); f_te

,단지코드,rent_per_depoist_ratio
0,C1003,0.01
1,C1006,0.03
2,C1016,0.01
3,C1019,0.01
4,C1030,0.00
...,...,...
145,C2653,0.01
146,C2675,0.01
147,C2676,0.02
148,C2688,0.01


##### [feature4]: 총세대수 대비 공가수 비율

In [170]:
f = (train.groupby('단지코드')['공가수'].agg([sum]) / train.groupby('단지코드')['총세대수'].agg([sum])).reset_index()
f = f.rename(columns = {'sum':'empty_ratio'})
f = f.fillna(0)
features.append(f); f

,단지코드,empty_ratio
0,C1000,0.02
1,C1004,0.01
2,C1005,0.01
3,C1013,0.01
4,C1014,0.01
...,...,...
406,C2663,0.02
407,C2666,0.02
408,C2670,0.01
409,C2680,0.03


In [171]:
f_te = (test.groupby('단지코드')['공가수'].agg([sum]) / test.groupby('단지코드')['총세대수'].agg([sum])).reset_index()
f_te = f_te.rename(columns = {'sum':'empty_ratio'})
f_te = f_te.fillna(0)
features_te.append(f_te); f_te

,단지코드,empty_ratio
0,C1003,0.06
1,C1006,0.02
2,C1016,0.01
3,C1019,0.02
4,C1030,0.16
...,...,...
145,C2653,0.03
146,C2675,0.02
147,C2676,0.01
148,C2688,0.01


##### [feature5]: 총세대수 대비 단지내주차면수 비율

In [172]:
f = (train.groupby('단지코드')['단지내주차면수'].agg([sum]) / train.groupby('단지코드')['총세대수'].agg([sum])).reset_index()
f = f.rename(columns = {'sum':'total_per_park_ratio'})
f = f.fillna(0)
features.append(f); f

,단지코드,total_per_park_ratio
0,C1000,0.77
1,C1004,0.29
2,C1005,0.83
3,C1013,0.86
4,C1014,0.83
...,...,...
406,C2663,0.91
407,C2666,0.77
408,C2670,0.93
409,C2680,0.74


In [173]:
f_te = (test.groupby('단지코드')['단지내주차면수'].agg([sum]) / test.groupby('단지코드')['총세대수'].agg([sum])).reset_index()
f_te = f_te.rename(columns = {'sum':'total_per_park_ratio'})
f_te = f_te.fillna(0)
features_te.append(f_te); f_te

,단지코드,total_per_park_ratio
0,C1003,0.71
1,C1006,0.28
2,C1016,1.00
3,C1019,0.81
4,C1030,0.39
...,...,...
145,C2653,0.91
146,C2675,1.98
147,C2676,0.16
148,C2688,0.71


##### [feature6]: 공가수 대비 단지내주차면수 비율

In [174]:
f = (train.groupby('단지코드')['단지내주차면수'].agg([sum]) / train.groupby('단지코드')['공가수'].agg([sum])).reset_index()
f = f.rename(columns = {'sum':'empty_per_park_ratio'})
f = f.fillna(0)
features.append(f); f

,단지코드,empty_per_park_ratio
0,C1000,43.80
1,C1004,51.00
2,C1005,59.38
3,C1013,69.94
4,C1014,164.60
...,...,...
406,C2663,50.00
407,C2666,36.39
408,C2670,66.67
409,C2680,29.68


In [175]:
f_te = (test.groupby('단지코드')['단지내주차면수'].agg([sum]) / test.groupby('단지코드')['공가수'].agg([sum])).reset_index()
f_te = f_te.rename(columns = {'sum':'empty_per_park_ratio'})
f_te = f_te.fillna(0)
features_te.append(f_te); f_te

,단지코드,empty_per_park_ratio
0,C1003,11.69
1,C1006,15.85
2,C1016,161.50
3,C1019,37.00
4,C1030,2.42
...,...,...
145,C2653,32.08
146,C2675,112.89
147,C2676,16.72
148,C2688,107.00


##### [feature7]: 전용면적 평균

##### [feature8]: 주차수요 추정방법 - 원단위법
- https://transpro.tistory.com/entry/%EC%A3%BC%EC%B0%A8%EC%88%98%EC%9A%94-%EC%B6%94%EC%A0%95%EB%B0%A9%EB%B2%95

### Categorical

In [176]:
object_columns = train.dtypes[train.dtypes == 'object'].index.tolist()
train[object_columns].head()

,단지코드,임대건물구분,지역,공급유형,자격유형
8,C2515,아파트,경상남도,국민임대/장기전세,A
9,C2515,아파트,경상남도,국민임대/장기전세,A
10,C2515,아파트,경상남도,국민임대/장기전세,A
11,C2515,아파트,경상남도,국민임대/장기전세,A
12,C2515,아파트,경상남도,국민임대/장기전세,A


In [177]:
# 나머지 카테고리는 위에서 이미 원핫인코딩함
# test '지역'에는 '서울특별시'가 없음...

f = reshape_cat_features(data=train, cast_col='지역', value_col='counter')
f = f.drop('지역_서울특별시', axis=1)
features.append(f); f

,지역_강원도,지역_경기도,지역_경상남도,지역_경상북도,지역_광주광역시,지역_대구광역시,지역_대전광역시,지역_부산광역시,지역_세종특별자치시,지역_울산광역시,지역_전라남도,지역_전라북도,지역_제주특별자치도,지역_충청남도,지역_충청북도
단지코드,,,,,,,,,,,,,,,
C1000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
C1004,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00
C1005,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00
C1013,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
C1014,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C2663,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
C2666,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
C2670,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00


In [178]:
f_te = reshape_cat_features(data=test, cast_col='지역', value_col='counter')
features_te.append(f_te); f_te

,지역_강원도,지역_경기도,지역_경상남도,지역_경상북도,지역_광주광역시,지역_대구광역시,지역_대전광역시,지역_부산광역시,지역_세종특별자치시,지역_울산광역시,지역_전라남도,지역_전라북도,지역_제주특별자치도,지역_충청남도,지역_충청북도
단지코드,,,,,,,,,,,,,,,
C1003,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
C1006,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
C1016,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
C1019,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
C1030,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C2653,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
C2675,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
C2676,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## Data merge

In [179]:
# 피쳐
data = pd.DataFrame({'단지코드': train.단지코드.unique()})

for f in features :
    data = pd.merge(data, f, how='left', on='단지코드')
    
data = data.fillna(0)

In [180]:
data_te = pd.DataFrame({'단지코드': test.단지코드.unique()})
for f in features_te :
    data_te = pd.merge(data_te, f, how='left', on='단지코드')
    
data_te = data_te.fillna(0)

In [181]:
# train_agg
data = pd.merge(train_agg, data, how='left', on='단지코드')
data_te = pd.merge(test_agg, data_te, how='left', on='단지코드')

In [182]:
# new_train
data = pd.merge(new_train, data, how='left', on='단지코드')
data_te = pd.merge(new_test, data_te, how='left', on='단지코드')

In [183]:
# agg_gender
data = pd.merge(data, age_gender, how='left', on='지역')
data_te = pd.merge(data_te, age_gender, how='left', on='지역')

In [184]:
data.head()

,단지코드,총세대수,공가수,지역,단지내주차면수,지하철,버스,면적_33.0,면적_39.0,면적_45.0,...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C2515,545.00,17.00,경상남도,624.00,0.00,3.00,276.00,80.00,57.00,...,0.09,0.07,0.05,0.03,0.03,0.01,0.00,0.00,0.00,0.00
1,C1407,1216.00,13.00,대전광역시,1285.00,1.00,1.00,0.00,292.00,170.00,...,0.09,0.07,0.05,0.04,0.03,0.01,0.01,0.00,0.00,0.00
2,C1945,755.00,6.00,경기도,734.00,1.00,3.00,0.00,120.00,0.00,...,0.07,0.06,0.04,0.03,0.03,0.01,0.01,0.00,0.00,0.00
3,C1470,696.00,14.00,전라북도,645.00,0.00,2.00,0.00,0.00,196.00,...,0.08,0.07,0.05,0.04,0.04,0.01,0.01,0.00,0.00,0.00
4,C1898,566.00,9.00,전라북도,517.00,0.00,6.00,0.00,0.00,209.00,...,0.08,0.07,0.05,0.04,0.04,0.01,0.01,0.00,0.00,0.00


In [185]:
data_te.head()

,단지코드,총세대수,공가수,지역,단지내주차면수,지하철,버스,면적_33.0,면적_39.0,면적_45.0,...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,C1072,754.00,14.00,경기도,683.00,0.00,2.00,0.00,116.00,262.00,...,0.07,0.06,0.04,0.03,0.03,0.01,0.01,0.00,0.00,0.00
1,C1128,1354.00,9.00,경기도,1216.00,0.00,3.00,0.00,420.00,356.00,...,0.07,0.06,0.04,0.03,0.03,0.01,0.01,0.00,0.00,0.00
2,C1456,619.00,18.00,부산광역시,547.00,0.00,16.00,82.00,165.00,110.00,...,0.11,0.09,0.08,0.05,0.05,0.02,0.01,0.00,0.00,0.00
3,C1840,593.00,7.00,전라북도,543.00,0.00,3.00,0.00,335.00,174.00,...,0.08,0.07,0.05,0.04,0.04,0.01,0.01,0.00,0.00,0.00
4,C1332,1297.00,11.00,경기도,1112.00,0.00,2.00,0.00,425.00,568.00,...,0.07,0.06,0.04,0.03,0.03,0.01,0.01,0.00,0.00,0.00


In [186]:
y_train = data['등록차량수']
data = data.drop('등록차량수', axis=1)

In [187]:
data = data.drop('지역', axis=1)
data_te = data_te.drop('지역', axis=1)

In [188]:
data.shape, data_te.shape, y_train.shape

((411, 96), (150, 96), (411,))

## Scaling

In [189]:
data = data.replace([np.inf, -np.inf], 0)
data_te = data_te.replace([np.inf, -np.inf], 0)

In [190]:
numeric_columns = data.dtypes[data.dtypes != 'object'].index.tolist()

In [191]:
# standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])
data_te[numeric_columns] = scaler.transform(data_te[numeric_columns])

## Modeling

In [192]:
data = data.values
data_te = data_te.values

In [193]:
ftr = data[:, 1:]
target = y_train
target_log = np.log1p(target)

In [194]:
from sklearn.model_selection import train_test_split

train_x, valid_x, train_y, valid_y = train_test_split(ftr, target, test_size=0.3, random_state=0)
train_x.shape, valid_x.shape

((287, 95), (124, 95))

In [195]:
from catboost import CatBoostRegressor

clf = CatBoostRegressor(
         n_estimators=500, 
         learning_rate=0.05, 
         random_state=42,
         loss_function = "MAE",
         eval_metric = 'MAE'
    )
    
clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)],  
        verbose=100, early_stopping_rounds= 50)

0:	learn: 270.0517410	test: 270.0517410	test1: 294.0987892	best: 294.0987892 (0)	total: 135ms	remaining: 1m 7s
100:	learn: 76.9063953	test: 76.9063953	test1: 147.4442932	best: 147.4442932 (100)	total: 427ms	remaining: 1.69s
200:	learn: 45.8003410	test: 45.8003410	test1: 135.0289955	best: 134.7676863 (196)	total: 584ms	remaining: 869ms
300:	learn: 33.6702285	test: 33.6702285	test1: 132.4333624	best: 132.4333624 (300)	total: 746ms	remaining: 494ms
400:	learn: 25.7316383	test: 25.7316383	test1: 130.7363016	best: 130.7327475 (397)	total: 986ms	remaining: 243ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 130.347585
bestIteration = 421

Shrink model to first 422 iterations.


In [196]:
#import matplotlib.pyplot as plt
#import seaborn as sns
#import matplotlib
#import matplotlib.font_manager as fm
#matplotlib.rcParams['axes.unicode_minus'] = False
#plt.rc('font', family=fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name())

#fea_imp = pd.DataFrame({'imp': clf.feature_importances_, 'col': X.columns})
#fea_imp = fea_imp.sort_values(['imp', 'col'], ascending=[True, False]).iloc[-30:]
#fea_imp.plot(kind='barh', x='col', y='imp', figsize=(10, 7), legend=None)
#plt.title('CatBoost - Feature Importance')
#plt.ylabel('Features')
#plt.xlabel('Importance');

##### Feature Selection & Cross Validation

In [197]:
ftr = data[:, 1:]
target = y_train
target_log = np.log1p(target)

In [198]:
from sklearn.feature_selection import SelectFromModel
smf = SelectFromModel(clf, threshold='1.0*mean')
smf.fit(ftr, target)
X_new = smf.transform(ftr)
X_te_new = smf.transform(data_te[:, 1:])
feature_idx = smf.get_support()
#feature_name = ftr.columns[feature_idx]

0:	learn: 277.3542568	total: 21.2ms	remaining: 10.6s
1:	learn: 269.6077178	total: 24.3ms	remaining: 6.04s
2:	learn: 260.4321835	total: 27.4ms	remaining: 4.55s
3:	learn: 253.2065502	total: 30ms	remaining: 3.72s
4:	learn: 245.4587654	total: 32.8ms	remaining: 3.24s
5:	learn: 239.5237770	total: 35.2ms	remaining: 2.9s
6:	learn: 233.4444581	total: 37.6ms	remaining: 2.65s
7:	learn: 227.3380207	total: 40.7ms	remaining: 2.5s
8:	learn: 222.1688934	total: 43.8ms	remaining: 2.39s
9:	learn: 216.3393310	total: 46.1ms	remaining: 2.26s
10:	learn: 210.9393796	total: 48.3ms	remaining: 2.15s
11:	learn: 206.9498321	total: 50.5ms	remaining: 2.05s
12:	learn: 202.5528593	total: 52.6ms	remaining: 1.97s
13:	learn: 198.3566744	total: 55ms	remaining: 1.91s
14:	learn: 193.9521158	total: 57.5ms	remaining: 1.86s
15:	learn: 190.6380123	total: 59.6ms	remaining: 1.8s
16:	learn: 187.1752600	total: 62.1ms	remaining: 1.76s
17:	learn: 182.9926321	total: 65.5ms	remaining: 1.75s
18:	learn: 178.9709685	total: 68.5ms	remainin

In [199]:
#ftr.columns[feature_idx]

In [200]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, X_new, target, scoring='neg_mean_absolute_error', cv=5)
print('교차 검증별 정확도:', np.round(scores, 4))
print('MAE:', -np.round(scores.mean(), 4))

# mean - 152.3205

0:	learn: 269.9236269	total: 14.9ms	remaining: 7.44s
1:	learn: 260.8469421	total: 16.8ms	remaining: 4.19s
2:	learn: 252.4430278	total: 19.8ms	remaining: 3.27s
3:	learn: 243.2833150	total: 22.3ms	remaining: 2.77s
4:	learn: 235.1032288	total: 24.1ms	remaining: 2.39s
5:	learn: 227.5801195	total: 26.8ms	remaining: 2.21s
6:	learn: 221.3561311	total: 28.3ms	remaining: 2s
7:	learn: 215.5912072	total: 29.8ms	remaining: 1.83s
8:	learn: 210.7522844	total: 31.2ms	remaining: 1.7s
9:	learn: 205.7678576	total: 32.7ms	remaining: 1.6s
10:	learn: 199.8768829	total: 34.2ms	remaining: 1.52s
11:	learn: 194.2121750	total: 35.7ms	remaining: 1.45s
12:	learn: 189.9378056	total: 37ms	remaining: 1.39s
13:	learn: 185.0384640	total: 38.5ms	remaining: 1.34s
14:	learn: 181.5038613	total: 40.4ms	remaining: 1.31s
15:	learn: 178.0864939	total: 42.4ms	remaining: 1.28s
16:	learn: 174.2839757	total: 44.9ms	remaining: 1.28s
17:	learn: 170.7273906	total: 47.1ms	remaining: 1.26s
18:	learn: 166.4437832	total: 48.8ms	remainin

In [201]:
X_new.shape, X_te_new.shape

((411, 21), (150, 21))

## Fitting

In [202]:
ftr = X_new
target = y_train
target_log = np.log1p(target)

In [203]:
from sklearn.model_selection import KFold

def train_apps_all_with_oof(ftr, target, nfolds=5):
    ftr = ftr
    target = target

    # nfolds 개의 cross validatin fold set을 가지는 KFold 생성 
    folds = KFold(n_splits=nfolds, shuffle=True, random_state=0)
    
    # Out of Folds로 학습된 모델의 validation set을 예측하여 결과 확률을 담을 array 생성.
    # validation set가 n_split갯수만큼 있으므로 크기는 ftr_app의 크기가 되어야 함. 
    oof_preds = np.zeros((ftr.shape[0],))  
    
    # Ouf of Folds로 학습된 모델의 test dataset을 예측하여 결과 확률을 담을 array 생성. 
    test_preds = np.zeros(((X_te_new.shape[0],)))

    # n_estimators를 4000까지 확대.
    clf = CatBoostRegressor(
             n_estimators=4000, 
             learning_rate=0.05, 
             random_state=42,
             loss_function = "MAE",
             eval_metric = 'MAE'
        )
    
    # nfolds 번 cross validation Iteration 반복하면서 OOF 방식으로 학습 및 테스트 데이터 예측
    for fold_idx, (train_idx, valid_idx) in enumerate(folds.split(ftr)):
        print('##### iteration ', fold_idx, ' 시작')
        # 학습용 데이터 세트의 인덱스와 검증용 데이터 세트의 인덱스 추출하여 이를 기반으로 학습/검증 데이터 추출
        train_x = ftr[train_idx, :]
        train_y = target[train_idx]
        valid_x = ftr[valid_idx, :]
        valid_y = target[valid_idx]
        
        # 추출된 학습/검증 데이터 세트로 모델 학습. early_stopping은 200으로 증가. 
        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], verbose= 200, 
                early_stopping_rounds= 200)
        # 검증 데이터 세트로 예측된 확률 저장. 사용되지는 않음. 
        #oof_preds[valid_idx] = clf.predict(valid_x, num_iteration=clf.best_iteration_)       
        # 학습된 모델로 테스트 데이터 세트에 예측 확률 계산. 
        # nfolds 번 반복 실행하므로 평균 확률을 구하기 위해 개별 수행시 마다 수행 횟수로 나눈 확률을 추후에 더해서 최종 평균 확률 계산. 
        test_preds += clf.predict(X_te_new)/folds.n_splits
        
        
    return clf, test_preds

In [204]:
clf, test_preds = train_apps_all_with_oof(ftr, target, nfolds=5)

##### iteration  0  시작
0:	learn: 274.9466452	test: 274.9466452	test1: 280.6126495	best: 280.6126495 (0)	total: 4.61ms	remaining: 18.4s
200:	learn: 49.3346098	test: 49.3346098	test1: 106.6676459	best: 106.1526909 (184)	total: 356ms	remaining: 6.74s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 106.1526909
bestIteration = 184

Shrink model to first 185 iterations.
##### iteration  1  시작
0:	learn: 263.9288742	test: 263.9288742	test1: 327.7189013	best: 327.7189013 (0)	total: 59.9ms	remaining: 3m 59s
200:	learn: 42.8694137	test: 42.8694137	test1: 160.4096597	best: 160.4096597 (200)	total: 437ms	remaining: 8.26s
400:	learn: 23.1261198	test: 23.1261198	test1: 158.3414676	best: 158.2727054 (379)	total: 759ms	remaining: 6.81s
600:	learn: 15.8890787	test: 15.8890787	test1: 157.2967584	best: 157.2353153 (560)	total: 1.14s	remaining: 6.47s
800:	learn: 12.4052374	test: 12.4052374	test1: 157.6281079	best: 156.6754678 (675)	total: 1.46s	remaining: 5.84s
Stopped by overfitting det

In [205]:
sub = pd.read_csv('../data/sample_submission.csv')
sub['num'] = (test_preds)
sub

,code,num
0,C1072,745.69
1,C1128,1222.79
2,C1456,544.19
3,C1840,528.34
4,C1332,1184.48
...,...,...
145,C2456,236.49
146,C1266,542.30
147,C2152,48.78
148,C1267,428.52


In [206]:
sub.to_csv('../submissions/sub14.csv', index=False)